In [1]:
'''
Author: ChenHJ
Date: 2022-12-06 17:46:00
LastEditors: ChenHJ
LastEditTime: 2022-12-06 17:49:38
FilePath: /chenhj/0302code/CMIP_divide_SASMR_plot_detrend.ipynb
Aim: 
12/06/2022:
  1) 对观测以及CMIP6的结果进行画图
1)
Mission: 
'''
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import *
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(sepl)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm

In [2]:
nIndia_N = 32.5
nIndia_S = 20.0
nIndia_W = 70.0
nIndia_E = 85.0

sIndia_N = 20.0
sIndia_S = 7.5
sIndia_W = 70.0
sIndia_E = 85.0

India_N = 32.5
India_S = 7.5
India_W = 70.0
India_E = 85.0

In [3]:
NC_N = 40.0
NC_S = 32.5
NC_W = 105.0
NC_E = 120.0

## 读取数据

### GPCP

In [4]:
nIndR_GPCP_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_GPCP_pre_regress.nc")
nIndR_ERA5_hgt_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_hgt_regress.nc")
nIndR_ERA5_u_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_u_regress.nc")
nIndR_ERA5_v_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_v_regress.nc")
nIndR_ERA5_wDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_wDJF_regress.nc")
nIndR_ERA5_wMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_wMAM_regress.nc")
nIndR_ERA5_wJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_wJJA_regress.nc")
nIndR_ERA5_wSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_wSON_regress.nc")
nIndR_ERA5_wnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_wnDJF_regress.nc")
nIndR_ERA5_div_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_div_regress.nc")

sIndR_GPCP_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_GPCP_pre_regress.nc")
sIndR_ERA5_hgt_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_hgt_regress.nc")
sIndR_ERA5_u_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_u_regress.nc")
sIndR_ERA5_v_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_v_regress.nc")
sIndR_ERA5_wDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_wDJF_regress.nc")
sIndR_ERA5_wMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_wMAM_regress.nc")
sIndR_ERA5_wJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_wJJA_regress.nc")
sIndR_ERA5_wSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_wSON_regress.nc")
sIndR_ERA5_wnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_wnDJF_regress.nc")
sIndR_ERA5_div_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_div_regress.nc")

nIndR_ERA5_sstDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_sstDJF_regress.nc")
nIndR_ERA5_sstMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_sstMAM_regress.nc")
nIndR_ERA5_sstJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_sstJJA_regress.nc")
nIndR_ERA5_sstSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_sstSON_regress.nc")
nIndR_ERA5_sstnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_sstnDJF_regress.nc")

sIndR_ERA5_sstDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_sstDJF_regress.nc")
sIndR_ERA5_sstMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_sstMAM_regress.nc")
sIndR_ERA5_sstJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_sstJJA_regress.nc")
sIndR_ERA5_sstSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_sstSON_regress.nc")
sIndR_ERA5_sstnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_sstnDJF_regress.nc")

nIndR_ERA5_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_uq_regress.nc")
nIndR_ERA5_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_vq_regress.nc")
nIndR_ERA5_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/GPCP/detrend/nIndR_ERA5_uqvqdiv_regress.nc")

sIndR_ERA5_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_uq_regress.nc")
sIndR_ERA5_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_vq_regress.nc")
sIndR_ERA5_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/GPCP/detrend/sIndR_ERA5_uqvqdiv_regress.nc")

IndR_GPCP_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_GPCP_pre_regress.nc")
IndR_ERA5_hgt_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_hgt_regress.nc")
IndR_ERA5_u_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_u_regress.nc")
IndR_ERA5_v_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_v_regress.nc")
IndR_ERA5_wDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_wDJF_regress.nc")
IndR_ERA5_wMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_wMAM_regress.nc")
IndR_ERA5_wJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_wJJA_regress.nc")
IndR_ERA5_wSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_wSON_regress.nc")
IndR_ERA5_wnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_wnDJF_regress.nc")
IndR_ERA5_div_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_div_regress.nc")

IndR_ERA5_sstDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_sstDJF_regress.nc")
IndR_ERA5_sstMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_sstMAM_regress.nc")
IndR_ERA5_sstJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_sstJJA_regress.nc")
IndR_ERA5_sstSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_sstSON_regress.nc")
IndR_ERA5_sstnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_sstnDJF_regress.nc")

IndR_ERA5_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_uq_regress.nc")
IndR_ERA5_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_vq_regress.nc")
IndR_ERA5_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/GPCP/detrend/IndR_ERA5_uqvqdiv_regress.nc")

### historical

In [5]:
nIndR_his_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_pre_regress.nc")
nIndR_his_hgt_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_hgt_regress.nc")
nIndR_his_u_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_u_regress.nc")
nIndR_his_v_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_v_regress.nc")
# nIndR_his_wDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_wDJF_regress.nc")
# nIndR_his_wMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_wMAM_regress.nc")
# nIndR_his_wJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_wJJA_regress.nc")
# nIndR_his_wSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_wSON_regress.nc")
# nIndR_his_wnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_wnDJF_regress.nc")
# nIndR_his_div_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_div_regress.nc")

sIndR_his_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_pre_regress.nc")
sIndR_his_hgt_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_hgt_regress.nc")
sIndR_his_u_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_u_regress.nc")
sIndR_his_v_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_v_regress.nc")
# sIndR_his_wDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_wDJF_regress.nc")
# sIndR_his_wMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_wMAM_regress.nc")
# sIndR_his_wJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_wJJA_regress.nc")
# sIndR_his_wSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_wSON_regress.nc")
# sIndR_his_wnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_wnDJF_regress.nc")
# sIndR_his_div_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_div_regress.nc")

nIndR_his_sstDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstDJF_regress.nc")
nIndR_his_sstMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstMAM_regress.nc")
nIndR_his_sstJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstJJA_regress.nc")
nIndR_his_sstSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstSON_regress.nc")
nIndR_his_sstnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_sstnDJF_regress.nc")

sIndR_his_sstDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstDJF_regress.nc")
sIndR_his_sstMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstMAM_regress.nc")
sIndR_his_sstJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstJJA_regress.nc")
sIndR_his_sstSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstSON_regress.nc")
sIndR_his_sstnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_sstnDJF_regress.nc")

nIndR_his_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_uq_regress.nc")
nIndR_his_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_vq_regress.nc")
nIndR_his_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_his_uqvqdiv_regress.nc")

sIndR_his_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_uq_regress.nc")
sIndR_his_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_vq_regress.nc")
sIndR_his_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_his_uqvqdiv_regress.nc")

IndR_his_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_pre_regress.nc")
IndR_his_hgt_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_hgt_regress.nc")
IndR_his_u_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_u_regress.nc")
IndR_his_v_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_v_regress.nc")
# IndR_his_wDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_wDJF_regress.nc")
# IndR_his_wMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_wMAM_regress.nc")
# IndR_his_wJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_wJJA_regress.nc")
# IndR_his_wSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_wSON_regress.nc")
# IndR_his_wnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_wnDJF_regress.nc")
# IndR_his_div_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_div_regress.nc")

IndR_his_sstDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstDJF_regress.nc")
IndR_his_sstMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstMAM_regress.nc")
IndR_his_sstJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstJJA_regress.nc")
IndR_his_sstSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstSON_regress.nc")
IndR_his_sstnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_sstnDJF_regress.nc")

IndR_his_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_uq_regress.nc")
IndR_his_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_vq_regress.nc")
IndR_his_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_his_uqvqdiv_regress.nc")

### ssp585

In [6]:
nIndR_ssp585_p3_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_pre_regress.nc")
nIndR_ssp585_p3_hgt_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_hgt_regress.nc")
nIndR_ssp585_p3_u_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_u_regress.nc")
nIndR_ssp585_p3_v_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_v_regress.nc")
# nIndR_ssp585_p3_wDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_wDJF_regress.nc")
# nIndR_ssp585_p3_wMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_wMAM_regress.nc")
# nIndR_ssp585_p3_wJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_wJJA_regress.nc")
# nIndR_ssp585_p3_wSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_wSON_regress.nc")
# nIndR_ssp585_p3_wnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_wnDJF_regress.nc")
# nIndR_ssp585_p3_div_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_div_regress.nc")

sIndR_ssp585_p3_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_pre_regress.nc")
sIndR_ssp585_p3_hgt_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_hgt_regress.nc")
sIndR_ssp585_p3_u_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_u_regress.nc")
sIndR_ssp585_p3_v_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_v_regress.nc")
# sIndR_ssp585_p3_wDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_wDJF_regress.nc")
# sIndR_ssp585_p3_wMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_wMAM_regress.nc")
# sIndR_ssp585_p3_wJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_wJJA_regress.nc")
# sIndR_ssp585_p3_wSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_wSON_regress.nc")
# sIndR_ssp585_p3_wnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_wnDJF_regress.nc")
# sIndR_ssp585_p3_div_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_div_regress.nc")

nIndR_ssp585_p3_sstDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstDJF_regress.nc")
nIndR_ssp585_p3_sstMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstMAM_regress.nc")
nIndR_ssp585_p3_sstJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstJJA_regress.nc")
nIndR_ssp585_p3_sstSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstSON_regress.nc")
nIndR_ssp585_p3_sstnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_sstnDJF_regress.nc")

sIndR_ssp585_p3_sstDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstDJF_regress.nc")
sIndR_ssp585_p3_sstMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstMAM_regress.nc")
sIndR_ssp585_p3_sstJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstJJA_regress.nc")
sIndR_ssp585_p3_sstSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstSON_regress.nc")
sIndR_ssp585_p3_sstnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_sstnDJF_regress.nc")

nIndR_ssp585_p3_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_uq_regress.nc")
nIndR_ssp585_p3_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_vq_regress.nc")
nIndR_ssp585_p3_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/NSASMR_reg_result/CMIP6/detrend/nIndR_ssp585_p3_uqvqdiv_regress.nc")

sIndR_ssp585_p3_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_uq_regress.nc")
sIndR_ssp585_p3_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_vq_regress.nc")
sIndR_ssp585_p3_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SSASMR_reg_result/CMIP6/detrend/sIndR_ssp585_p3_uqvqdiv_regress.nc")

IndR_ssp585_p3_pre_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_pre_regress.nc")
IndR_ssp585_p3_hgt_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_hgt_regress.nc")
IndR_ssp585_p3_u_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_u_regress.nc")
IndR_ssp585_p3_v_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_v_regress.nc")
# IndR_ssp585_p3_wDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_wDJF_regress.nc")
# IndR_ssp585_p3_wMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_wMAM_regress.nc")
# IndR_ssp585_p3_wJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_wJJA_regress.nc")
# IndR_ssp585_p3_wSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_wSON_regress.nc")
# IndR_ssp585_p3_wnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_wnDJF_regress.nc")
# IndR_ssp585_p3_div_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_div_regress.nc")

IndR_ssp585_p3_sstDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstDJF_regress.nc")
IndR_ssp585_p3_sstMAM_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstMAM_regress.nc")
IndR_ssp585_p3_sstJJA_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstJJA_regress.nc")
IndR_ssp585_p3_sstSON_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstSON_regress.nc")
IndR_ssp585_p3_sstnDJF_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_sstnDJF_regress.nc")

IndR_ssp585_p3_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_uq_regress.nc")
IndR_ssp585_p3_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_vq_regress.nc")
IndR_ssp585_p3_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/CMIP6/detrend/IndR_ssp585_p3_uqvqdiv_regress.nc")

In [7]:
models = IndR_his_hgt_regress.coords["models"]

### 读取回归计算得到的数据

#### 观测

In [8]:
nIndR_GPCP_pre_slope = nIndR_GPCP_pre_regress["slope"]
nIndR_GPCP_pre_rvalue = nIndR_GPCP_pre_regress["rvalue"]
nIndR_GPCP_pre_pvalue = nIndR_GPCP_pre_regress["pvalue"]

nIndR_ERA5_hgt_slope = nIndR_ERA5_hgt_regress["slope"]
nIndR_ERA5_hgt_rvalue = nIndR_ERA5_hgt_regress["rvalue"]
nIndR_ERA5_hgt_pvalue = nIndR_ERA5_hgt_regress["pvalue"]

nIndR_ERA5_u_slope = nIndR_ERA5_u_regress["slope"]
nIndR_ERA5_u_rvalue = nIndR_ERA5_u_regress["rvalue"]
nIndR_ERA5_u_pvalue = nIndR_ERA5_u_regress["pvalue"]

nIndR_ERA5_v_slope = nIndR_ERA5_v_regress["slope"]
nIndR_ERA5_v_rvalue = nIndR_ERA5_v_regress["rvalue"]
nIndR_ERA5_v_pvalue = nIndR_ERA5_v_regress["pvalue"]

nIndR_ERA5_wDJF_slope = nIndR_ERA5_wDJF_regress["slope"]
nIndR_ERA5_wDJF_rvalue = nIndR_ERA5_wDJF_regress["rvalue"]
nIndR_ERA5_wDJF_pvalue = nIndR_ERA5_wDJF_regress["pvalue"]

nIndR_ERA5_wMAM_slope = nIndR_ERA5_wMAM_regress["slope"]
nIndR_ERA5_wMAM_rvalue = nIndR_ERA5_wMAM_regress["rvalue"]
nIndR_ERA5_wMAM_pvalue = nIndR_ERA5_wMAM_regress["pvalue"]

nIndR_ERA5_wJJA_slope = nIndR_ERA5_wJJA_regress["slope"]
nIndR_ERA5_wJJA_rvalue = nIndR_ERA5_wJJA_regress["rvalue"]
nIndR_ERA5_wJJA_pvalue = nIndR_ERA5_wJJA_regress["pvalue"]

nIndR_ERA5_wSON_slope = nIndR_ERA5_wSON_regress["slope"]
nIndR_ERA5_wSON_rvalue = nIndR_ERA5_wSON_regress["rvalue"]
nIndR_ERA5_wSON_pvalue = nIndR_ERA5_wSON_regress["pvalue"]

nIndR_ERA5_wnDJF_slope = nIndR_ERA5_wnDJF_regress["slope"]
nIndR_ERA5_wnDJF_rvalue = nIndR_ERA5_wnDJF_regress["rvalue"]
nIndR_ERA5_wnDJF_pvalue = nIndR_ERA5_wnDJF_regress["pvalue"]

nIndR_ERA5_div_slope = nIndR_ERA5_div_regress["slope"]
nIndR_ERA5_div_rvalue = nIndR_ERA5_div_regress["rvalue"]
nIndR_ERA5_div_pvalue = nIndR_ERA5_div_regress["pvalue"]

nIndR_ERA5_sstDJF_slope = nIndR_ERA5_sstDJF_regress["slope"]
nIndR_ERA5_sstDJF_rvalue = nIndR_ERA5_sstDJF_regress["rvalue"]
nIndR_ERA5_sstDJF_pvalue = nIndR_ERA5_sstDJF_regress["pvalue"]

nIndR_ERA5_sstMAM_slope = nIndR_ERA5_sstMAM_regress["slope"]
nIndR_ERA5_sstMAM_rvalue = nIndR_ERA5_sstMAM_regress["rvalue"]
nIndR_ERA5_sstMAM_pvalue = nIndR_ERA5_sstMAM_regress["pvalue"]

nIndR_ERA5_sstJJA_slope = nIndR_ERA5_sstJJA_regress["slope"]
nIndR_ERA5_sstJJA_rvalue = nIndR_ERA5_sstJJA_regress["rvalue"]
nIndR_ERA5_sstJJA_pvalue = nIndR_ERA5_sstJJA_regress["pvalue"]

nIndR_ERA5_sstSON_slope = nIndR_ERA5_sstSON_regress["slope"]
nIndR_ERA5_sstSON_rvalue = nIndR_ERA5_sstSON_regress["rvalue"]
nIndR_ERA5_sstSON_pvalue = nIndR_ERA5_sstSON_regress["pvalue"]

nIndR_ERA5_sstnDJF_slope = nIndR_ERA5_sstnDJF_regress["slope"]
nIndR_ERA5_sstnDJF_rvalue = nIndR_ERA5_sstnDJF_regress["rvalue"]
nIndR_ERA5_sstnDJF_pvalue = nIndR_ERA5_sstnDJF_regress["pvalue"]

nIndR_ERA5_uq_slope = nIndR_ERA5_uq_regress["slope"]
nIndR_ERA5_uq_rvalue = nIndR_ERA5_uq_regress["rvalue"]
nIndR_ERA5_uq_pvalue = nIndR_ERA5_uq_regress["pvalue"]

nIndR_ERA5_vq_slope = nIndR_ERA5_vq_regress["slope"]
nIndR_ERA5_vq_rvalue = nIndR_ERA5_vq_regress["rvalue"]
nIndR_ERA5_vq_pvalue = nIndR_ERA5_vq_regress["pvalue"]

nIndR_ERA5_uqvqdiv_slope = nIndR_ERA5_uqvqdiv_regress["slope"]
nIndR_ERA5_uqvqdiv_rvalue = nIndR_ERA5_uqvqdiv_regress["rvalue"]
nIndR_ERA5_uqvqdiv_pvalue = nIndR_ERA5_uqvqdiv_regress["pvalue"]

In [9]:
sIndR_GPCP_pre_slope = sIndR_GPCP_pre_regress["slope"]
sIndR_GPCP_pre_rvalue = sIndR_GPCP_pre_regress["rvalue"]
sIndR_GPCP_pre_pvalue = sIndR_GPCP_pre_regress["pvalue"]

sIndR_ERA5_hgt_slope = sIndR_ERA5_hgt_regress["slope"]
sIndR_ERA5_hgt_rvalue = sIndR_ERA5_hgt_regress["rvalue"]
sIndR_ERA5_hgt_pvalue = sIndR_ERA5_hgt_regress["pvalue"]

sIndR_ERA5_u_slope = sIndR_ERA5_u_regress["slope"]
sIndR_ERA5_u_rvalue = sIndR_ERA5_u_regress["rvalue"]
sIndR_ERA5_u_pvalue = sIndR_ERA5_u_regress["pvalue"]

sIndR_ERA5_v_slope = sIndR_ERA5_v_regress["slope"]
sIndR_ERA5_v_rvalue = sIndR_ERA5_v_regress["rvalue"]
sIndR_ERA5_v_pvalue = sIndR_ERA5_v_regress["pvalue"]

sIndR_ERA5_wDJF_slope = sIndR_ERA5_wDJF_regress["slope"]
sIndR_ERA5_wDJF_rvalue = sIndR_ERA5_wDJF_regress["rvalue"]
sIndR_ERA5_wDJF_pvalue = sIndR_ERA5_wDJF_regress["pvalue"]

sIndR_ERA5_wMAM_slope = sIndR_ERA5_wMAM_regress["slope"]
sIndR_ERA5_wMAM_rvalue = sIndR_ERA5_wMAM_regress["rvalue"]
sIndR_ERA5_wMAM_pvalue = sIndR_ERA5_wMAM_regress["pvalue"]

sIndR_ERA5_wJJA_slope = sIndR_ERA5_wJJA_regress["slope"]
sIndR_ERA5_wJJA_rvalue = sIndR_ERA5_wJJA_regress["rvalue"]
sIndR_ERA5_wJJA_pvalue = sIndR_ERA5_wJJA_regress["pvalue"]

sIndR_ERA5_wSON_slope = sIndR_ERA5_wSON_regress["slope"]
sIndR_ERA5_wSON_rvalue = sIndR_ERA5_wSON_regress["rvalue"]
sIndR_ERA5_wSON_pvalue = sIndR_ERA5_wSON_regress["pvalue"]

sIndR_ERA5_wnDJF_slope = sIndR_ERA5_wnDJF_regress["slope"]
sIndR_ERA5_wnDJF_rvalue = sIndR_ERA5_wnDJF_regress["rvalue"]
sIndR_ERA5_wnDJF_pvalue = sIndR_ERA5_wnDJF_regress["pvalue"]

sIndR_ERA5_div_slope = sIndR_ERA5_div_regress["slope"]
sIndR_ERA5_div_rvalue = sIndR_ERA5_div_regress["rvalue"]
sIndR_ERA5_div_pvalue = sIndR_ERA5_div_regress["pvalue"]

sIndR_ERA5_sstDJF_slope = sIndR_ERA5_sstDJF_regress["slope"]
sIndR_ERA5_sstDJF_rvalue = sIndR_ERA5_sstDJF_regress["rvalue"]
sIndR_ERA5_sstDJF_pvalue = sIndR_ERA5_sstDJF_regress["pvalue"]

sIndR_ERA5_sstMAM_slope = sIndR_ERA5_sstMAM_regress["slope"]
sIndR_ERA5_sstMAM_rvalue = sIndR_ERA5_sstMAM_regress["rvalue"]
sIndR_ERA5_sstMAM_pvalue = sIndR_ERA5_sstMAM_regress["pvalue"]

sIndR_ERA5_sstJJA_slope = sIndR_ERA5_sstJJA_regress["slope"]
sIndR_ERA5_sstJJA_rvalue = sIndR_ERA5_sstJJA_regress["rvalue"]
sIndR_ERA5_sstJJA_pvalue = sIndR_ERA5_sstJJA_regress["pvalue"]

sIndR_ERA5_sstSON_slope = sIndR_ERA5_sstSON_regress["slope"]
sIndR_ERA5_sstSON_rvalue = sIndR_ERA5_sstSON_regress["rvalue"]
sIndR_ERA5_sstSON_pvalue = sIndR_ERA5_sstSON_regress["pvalue"]

sIndR_ERA5_sstnDJF_slope = sIndR_ERA5_sstnDJF_regress["slope"]
sIndR_ERA5_sstnDJF_rvalue = sIndR_ERA5_sstnDJF_regress["rvalue"]
sIndR_ERA5_sstnDJF_pvalue = sIndR_ERA5_sstnDJF_regress["pvalue"]

sIndR_ERA5_uq_slope = sIndR_ERA5_uq_regress["slope"]
sIndR_ERA5_uq_rvalue = sIndR_ERA5_uq_regress["rvalue"]
sIndR_ERA5_uq_pvalue = sIndR_ERA5_uq_regress["pvalue"]

sIndR_ERA5_vq_slope = sIndR_ERA5_vq_regress["slope"]
sIndR_ERA5_vq_rvalue = sIndR_ERA5_vq_regress["rvalue"]
sIndR_ERA5_vq_pvalue = sIndR_ERA5_vq_regress["pvalue"]

sIndR_ERA5_uqvqdiv_slope = sIndR_ERA5_uqvqdiv_regress["slope"]
sIndR_ERA5_uqvqdiv_rvalue = sIndR_ERA5_uqvqdiv_regress["rvalue"]
sIndR_ERA5_uqvqdiv_pvalue = sIndR_ERA5_uqvqdiv_regress["pvalue"]

In [10]:
IndR_GPCP_pre_slope = IndR_GPCP_pre_regress["slope"]
IndR_GPCP_pre_rvalue = IndR_GPCP_pre_regress["rvalue"]
IndR_GPCP_pre_pvalue = IndR_GPCP_pre_regress["pvalue"]

IndR_ERA5_hgt_slope = IndR_ERA5_hgt_regress["slope"]
IndR_ERA5_hgt_rvalue = IndR_ERA5_hgt_regress["rvalue"]
IndR_ERA5_hgt_pvalue = IndR_ERA5_hgt_regress["pvalue"]

IndR_ERA5_u_slope = IndR_ERA5_u_regress["slope"]
IndR_ERA5_u_rvalue = IndR_ERA5_u_regress["rvalue"]
IndR_ERA5_u_pvalue = IndR_ERA5_u_regress["pvalue"]

IndR_ERA5_v_slope = IndR_ERA5_v_regress["slope"]
IndR_ERA5_v_rvalue = IndR_ERA5_v_regress["rvalue"]
IndR_ERA5_v_pvalue = IndR_ERA5_v_regress["pvalue"]

IndR_ERA5_wDJF_slope = IndR_ERA5_wDJF_regress["slope"]
IndR_ERA5_wDJF_rvalue = IndR_ERA5_wDJF_regress["rvalue"]
IndR_ERA5_wDJF_pvalue = IndR_ERA5_wDJF_regress["pvalue"]

IndR_ERA5_wMAM_slope = IndR_ERA5_wMAM_regress["slope"]
IndR_ERA5_wMAM_rvalue = IndR_ERA5_wMAM_regress["rvalue"]
IndR_ERA5_wMAM_pvalue = IndR_ERA5_wMAM_regress["pvalue"]

IndR_ERA5_wJJA_slope = IndR_ERA5_wJJA_regress["slope"]
IndR_ERA5_wJJA_rvalue = IndR_ERA5_wJJA_regress["rvalue"]
IndR_ERA5_wJJA_pvalue = IndR_ERA5_wJJA_regress["pvalue"]

IndR_ERA5_wSON_slope = IndR_ERA5_wSON_regress["slope"]
IndR_ERA5_wSON_rvalue = IndR_ERA5_wSON_regress["rvalue"]
IndR_ERA5_wSON_pvalue = IndR_ERA5_wSON_regress["pvalue"]

IndR_ERA5_wnDJF_slope = IndR_ERA5_wnDJF_regress["slope"]
IndR_ERA5_wnDJF_rvalue = IndR_ERA5_wnDJF_regress["rvalue"]
IndR_ERA5_wnDJF_pvalue = IndR_ERA5_wnDJF_regress["pvalue"]

IndR_ERA5_div_slope = IndR_ERA5_div_regress["slope"]
IndR_ERA5_div_rvalue = IndR_ERA5_div_regress["rvalue"]
IndR_ERA5_div_pvalue = IndR_ERA5_div_regress["pvalue"]

IndR_ERA5_sstDJF_slope = IndR_ERA5_sstDJF_regress["slope"]
IndR_ERA5_sstDJF_rvalue = IndR_ERA5_sstDJF_regress["rvalue"]
IndR_ERA5_sstDJF_pvalue = IndR_ERA5_sstDJF_regress["pvalue"]

IndR_ERA5_sstMAM_slope = IndR_ERA5_sstMAM_regress["slope"]
IndR_ERA5_sstMAM_rvalue = IndR_ERA5_sstMAM_regress["rvalue"]
IndR_ERA5_sstMAM_pvalue = IndR_ERA5_sstMAM_regress["pvalue"]

IndR_ERA5_sstJJA_slope = IndR_ERA5_sstJJA_regress["slope"]
IndR_ERA5_sstJJA_rvalue = IndR_ERA5_sstJJA_regress["rvalue"]
IndR_ERA5_sstJJA_pvalue = IndR_ERA5_sstJJA_regress["pvalue"]

IndR_ERA5_sstSON_slope = IndR_ERA5_sstSON_regress["slope"]
IndR_ERA5_sstSON_rvalue = IndR_ERA5_sstSON_regress["rvalue"]
IndR_ERA5_sstSON_pvalue = IndR_ERA5_sstSON_regress["pvalue"]

IndR_ERA5_sstnDJF_slope = IndR_ERA5_sstnDJF_regress["slope"]
IndR_ERA5_sstnDJF_rvalue = IndR_ERA5_sstnDJF_regress["rvalue"]
IndR_ERA5_sstnDJF_pvalue = IndR_ERA5_sstnDJF_regress["pvalue"]

IndR_ERA5_uq_slope = IndR_ERA5_uq_regress["slope"]
IndR_ERA5_uq_rvalue = IndR_ERA5_uq_regress["rvalue"]
IndR_ERA5_uq_pvalue = IndR_ERA5_uq_regress["pvalue"]

IndR_ERA5_vq_slope = IndR_ERA5_vq_regress["slope"]
IndR_ERA5_vq_rvalue = IndR_ERA5_vq_regress["rvalue"]
IndR_ERA5_vq_pvalue = IndR_ERA5_vq_regress["pvalue"]

IndR_ERA5_uqvqdiv_slope = IndR_ERA5_uqvqdiv_regress["slope"]
IndR_ERA5_uqvqdiv_rvalue = IndR_ERA5_uqvqdiv_regress["rvalue"]
IndR_ERA5_uqvqdiv_pvalue = IndR_ERA5_uqvqdiv_regress["pvalue"]

#### historical

In [11]:
nIndR_his_pre_slope = nIndR_his_pre_regress["slope"]
nIndR_his_pre_rvalue = nIndR_his_pre_regress["rvalue"]
nIndR_his_pre_pvalue = nIndR_his_pre_regress["pvalue"]

nIndR_his_hgt_slope = nIndR_his_hgt_regress["slope"]
nIndR_his_hgt_rvalue = nIndR_his_hgt_regress["rvalue"]
nIndR_his_hgt_pvalue = nIndR_his_hgt_regress["pvalue"]

nIndR_his_u_slope = nIndR_his_u_regress["slope"]
nIndR_his_u_rvalue = nIndR_his_u_regress["rvalue"]
nIndR_his_u_pvalue = nIndR_his_u_regress["pvalue"]

nIndR_his_v_slope = nIndR_his_v_regress["slope"]
nIndR_his_v_rvalue = nIndR_his_v_regress["rvalue"]
nIndR_his_v_pvalue = nIndR_his_v_regress["pvalue"]

# nIndR_his_wDJF_slope = nIndR_his_wDJF_regress["slope"]
# nIndR_his_wDJF_rvalue = nIndR_his_wDJF_regress["rvalue"]
# nIndR_his_wDJF_pvalue = nIndR_his_wDJF_regress["pvalue"]

# nIndR_his_wMAM_slope = nIndR_his_wMAM_regress["slope"]
# nIndR_his_wMAM_rvalue = nIndR_his_wMAM_regress["rvalue"]
# nIndR_his_wMAM_pvalue = nIndR_his_wMAM_regress["pvalue"]

# nIndR_his_wJJA_slope = nIndR_his_wJJA_regress["slope"]
# nIndR_his_wJJA_rvalue = nIndR_his_wJJA_regress["rvalue"]
# nIndR_his_wJJA_pvalue = nIndR_his_wJJA_regress["pvalue"]

# nIndR_his_wSON_slope = nIndR_his_wSON_regress["slope"]
# nIndR_his_wSON_rvalue = nIndR_his_wSON_regress["rvalue"]
# nIndR_his_wSON_pvalue = nIndR_his_wSON_regress["pvalue"]

# nIndR_his_wnDJF_slope = nIndR_his_wnDJF_regress["slope"]
# nIndR_his_wnDJF_rvalue = nIndR_his_wnDJF_regress["rvalue"]
# nIndR_his_wnDJF_pvalue = nIndR_his_wnDJF_regress["pvalue"]

# nIndR_his_div_slope = nIndR_his_div_regress["slope"]
# nIndR_his_div_rvalue = nIndR_his_div_regress["rvalue"]
# nIndR_his_div_pvalue = nIndR_his_div_regress["pvalue"]

nIndR_his_sstDJF_slope = nIndR_his_sstDJF_regress["slope"]
nIndR_his_sstDJF_rvalue = nIndR_his_sstDJF_regress["rvalue"]
nIndR_his_sstDJF_pvalue = nIndR_his_sstDJF_regress["pvalue"]

nIndR_his_sstMAM_slope = nIndR_his_sstMAM_regress["slope"]
nIndR_his_sstMAM_rvalue = nIndR_his_sstMAM_regress["rvalue"]
nIndR_his_sstMAM_pvalue = nIndR_his_sstMAM_regress["pvalue"]

nIndR_his_sstJJA_slope = nIndR_his_sstJJA_regress["slope"]
nIndR_his_sstJJA_rvalue = nIndR_his_sstJJA_regress["rvalue"]
nIndR_his_sstJJA_pvalue = nIndR_his_sstJJA_regress["pvalue"]

nIndR_his_sstSON_slope = nIndR_his_sstSON_regress["slope"]
nIndR_his_sstSON_rvalue = nIndR_his_sstSON_regress["rvalue"]
nIndR_his_sstSON_pvalue = nIndR_his_sstSON_regress["pvalue"]

nIndR_his_sstnDJF_slope = nIndR_his_sstnDJF_regress["slope"]
nIndR_his_sstnDJF_rvalue = nIndR_his_sstnDJF_regress["rvalue"]
nIndR_his_sstnDJF_pvalue = nIndR_his_sstnDJF_regress["pvalue"]

nIndR_his_uq_slope = nIndR_his_uq_regress["slope"]
nIndR_his_uq_rvalue = nIndR_his_uq_regress["rvalue"]
nIndR_his_uq_pvalue = nIndR_his_uq_regress["pvalue"]

nIndR_his_vq_slope = nIndR_his_vq_regress["slope"]
nIndR_his_vq_rvalue = nIndR_his_vq_regress["rvalue"]
nIndR_his_vq_pvalue = nIndR_his_vq_regress["pvalue"]

nIndR_his_uqvqdiv_slope = nIndR_his_uqvqdiv_regress["slope"]
nIndR_his_uqvqdiv_rvalue = nIndR_his_uqvqdiv_regress["rvalue"]
nIndR_his_uqvqdiv_pvalue = nIndR_his_uqvqdiv_regress["pvalue"]

In [12]:
sIndR_his_pre_slope = sIndR_his_pre_regress["slope"]
sIndR_his_pre_rvalue = sIndR_his_pre_regress["rvalue"]
sIndR_his_pre_pvalue = sIndR_his_pre_regress["pvalue"]

sIndR_his_hgt_slope = sIndR_his_hgt_regress["slope"]
sIndR_his_hgt_rvalue = sIndR_his_hgt_regress["rvalue"]
sIndR_his_hgt_pvalue = sIndR_his_hgt_regress["pvalue"]

sIndR_his_u_slope = sIndR_his_u_regress["slope"]
sIndR_his_u_rvalue = sIndR_his_u_regress["rvalue"]
sIndR_his_u_pvalue = sIndR_his_u_regress["pvalue"]

sIndR_his_v_slope = sIndR_his_v_regress["slope"]
sIndR_his_v_rvalue = sIndR_his_v_regress["rvalue"]
sIndR_his_v_pvalue = sIndR_his_v_regress["pvalue"]

# sIndR_his_wDJF_slope = sIndR_his_wDJF_regress["slope"]
# sIndR_his_wDJF_rvalue = sIndR_his_wDJF_regress["rvalue"]
# sIndR_his_wDJF_pvalue = sIndR_his_wDJF_regress["pvalue"]

# sIndR_his_wMAM_slope = sIndR_his_wMAM_regress["slope"]
# sIndR_his_wMAM_rvalue = sIndR_his_wMAM_regress["rvalue"]
# sIndR_his_wMAM_pvalue = sIndR_his_wMAM_regress["pvalue"]

# sIndR_his_wJJA_slope = sIndR_his_wJJA_regress["slope"]
# sIndR_his_wJJA_rvalue = sIndR_his_wJJA_regress["rvalue"]
# sIndR_his_wJJA_pvalue = sIndR_his_wJJA_regress["pvalue"]

# sIndR_his_wSON_slope = sIndR_his_wSON_regress["slope"]
# sIndR_his_wSON_rvalue = sIndR_his_wSON_regress["rvalue"]
# sIndR_his_wSON_pvalue = sIndR_his_wSON_regress["pvalue"]

# sIndR_his_wnDJF_slope = sIndR_his_wnDJF_regress["slope"]
# sIndR_his_wnDJF_rvalue = sIndR_his_wnDJF_regress["rvalue"]
# sIndR_his_wnDJF_pvalue = sIndR_his_wnDJF_regress["pvalue"]

# sIndR_his_div_slope = sIndR_his_div_regress["slope"]
# sIndR_his_div_rvalue = sIndR_his_div_regress["rvalue"]
# sIndR_his_div_pvalue = sIndR_his_div_regress["pvalue"]

sIndR_his_sstDJF_slope = sIndR_his_sstDJF_regress["slope"]
sIndR_his_sstDJF_rvalue = sIndR_his_sstDJF_regress["rvalue"]
sIndR_his_sstDJF_pvalue = sIndR_his_sstDJF_regress["pvalue"]

sIndR_his_sstMAM_slope = sIndR_his_sstMAM_regress["slope"]
sIndR_his_sstMAM_rvalue = sIndR_his_sstMAM_regress["rvalue"]
sIndR_his_sstMAM_pvalue = sIndR_his_sstMAM_regress["pvalue"]

sIndR_his_sstJJA_slope = sIndR_his_sstJJA_regress["slope"]
sIndR_his_sstJJA_rvalue = sIndR_his_sstJJA_regress["rvalue"]
sIndR_his_sstJJA_pvalue = sIndR_his_sstJJA_regress["pvalue"]

sIndR_his_sstSON_slope = sIndR_his_sstSON_regress["slope"]
sIndR_his_sstSON_rvalue = sIndR_his_sstSON_regress["rvalue"]
sIndR_his_sstSON_pvalue = sIndR_his_sstSON_regress["pvalue"]

sIndR_his_sstnDJF_slope = sIndR_his_sstnDJF_regress["slope"]
sIndR_his_sstnDJF_rvalue = sIndR_his_sstnDJF_regress["rvalue"]
sIndR_his_sstnDJF_pvalue = sIndR_his_sstnDJF_regress["pvalue"]

sIndR_his_uq_slope = sIndR_his_uq_regress["slope"]
sIndR_his_uq_rvalue = sIndR_his_uq_regress["rvalue"]
sIndR_his_uq_pvalue = sIndR_his_uq_regress["pvalue"]

sIndR_his_vq_slope = sIndR_his_vq_regress["slope"]
sIndR_his_vq_rvalue = sIndR_his_vq_regress["rvalue"]
sIndR_his_vq_pvalue = sIndR_his_vq_regress["pvalue"]

sIndR_his_uqvqdiv_slope = sIndR_his_uqvqdiv_regress["slope"]
sIndR_his_uqvqdiv_rvalue = sIndR_his_uqvqdiv_regress["rvalue"]
sIndR_his_uqvqdiv_pvalue = sIndR_his_uqvqdiv_regress["pvalue"]

In [13]:
IndR_his_pre_slope = IndR_his_pre_regress["slope"]
IndR_his_pre_rvalue = IndR_his_pre_regress["rvalue"]
IndR_his_pre_pvalue = IndR_his_pre_regress["pvalue"]

IndR_his_hgt_slope = IndR_his_hgt_regress["slope"]
IndR_his_hgt_rvalue = IndR_his_hgt_regress["rvalue"]
IndR_his_hgt_pvalue = IndR_his_hgt_regress["pvalue"]

IndR_his_u_slope = IndR_his_u_regress["slope"]
IndR_his_u_rvalue = IndR_his_u_regress["rvalue"]
IndR_his_u_pvalue = IndR_his_u_regress["pvalue"]

IndR_his_v_slope = IndR_his_v_regress["slope"]
IndR_his_v_rvalue = IndR_his_v_regress["rvalue"]
IndR_his_v_pvalue = IndR_his_v_regress["pvalue"]

# IndR_his_wDJF_slope = IndR_his_wDJF_regress["slope"]
# IndR_his_wDJF_rvalue = IndR_his_wDJF_regress["rvalue"]
# IndR_his_wDJF_pvalue = IndR_his_wDJF_regress["pvalue"]

# IndR_his_wMAM_slope = IndR_his_wMAM_regress["slope"]
# IndR_his_wMAM_rvalue = IndR_his_wMAM_regress["rvalue"]
# IndR_his_wMAM_pvalue = IndR_his_wMAM_regress["pvalue"]

# IndR_his_wJJA_slope = IndR_his_wJJA_regress["slope"]
# IndR_his_wJJA_rvalue = IndR_his_wJJA_regress["rvalue"]
# IndR_his_wJJA_pvalue = IndR_his_wJJA_regress["pvalue"]

# IndR_his_wSON_slope = IndR_his_wSON_regress["slope"]
# IndR_his_wSON_rvalue = IndR_his_wSON_regress["rvalue"]
# IndR_his_wSON_pvalue = IndR_his_wSON_regress["pvalue"]

# IndR_his_wnDJF_slope = IndR_his_wnDJF_regress["slope"]
# IndR_his_wnDJF_rvalue = IndR_his_wnDJF_regress["rvalue"]
# IndR_his_wnDJF_pvalue = IndR_his_wnDJF_regress["pvalue"]

# IndR_his_div_slope = IndR_his_div_regress["slope"]
# IndR_his_div_rvalue = IndR_his_div_regress["rvalue"]
# IndR_his_div_pvalue = IndR_his_div_regress["pvalue"]

IndR_his_sstDJF_slope = IndR_his_sstDJF_regress["slope"]
IndR_his_sstDJF_rvalue = IndR_his_sstDJF_regress["rvalue"]
IndR_his_sstDJF_pvalue = IndR_his_sstDJF_regress["pvalue"]

IndR_his_sstMAM_slope = IndR_his_sstMAM_regress["slope"]
IndR_his_sstMAM_rvalue = IndR_his_sstMAM_regress["rvalue"]
IndR_his_sstMAM_pvalue = IndR_his_sstMAM_regress["pvalue"]

IndR_his_sstJJA_slope = IndR_his_sstJJA_regress["slope"]
IndR_his_sstJJA_rvalue = IndR_his_sstJJA_regress["rvalue"]
IndR_his_sstJJA_pvalue = IndR_his_sstJJA_regress["pvalue"]

IndR_his_sstSON_slope = IndR_his_sstSON_regress["slope"]
IndR_his_sstSON_rvalue = IndR_his_sstSON_regress["rvalue"]
IndR_his_sstSON_pvalue = IndR_his_sstSON_regress["pvalue"]

IndR_his_sstnDJF_slope = IndR_his_sstnDJF_regress["slope"]
IndR_his_sstnDJF_rvalue = IndR_his_sstnDJF_regress["rvalue"]
IndR_his_sstnDJF_pvalue = IndR_his_sstnDJF_regress["pvalue"]

IndR_his_uq_slope = IndR_his_uq_regress["slope"]
IndR_his_uq_rvalue = IndR_his_uq_regress["rvalue"]
IndR_his_uq_pvalue = IndR_his_uq_regress["pvalue"]

IndR_his_vq_slope = IndR_his_vq_regress["slope"]
IndR_his_vq_rvalue = IndR_his_vq_regress["rvalue"]
IndR_his_vq_pvalue = IndR_his_vq_regress["pvalue"]

IndR_his_uqvqdiv_slope = IndR_his_uqvqdiv_regress["slope"]
IndR_his_uqvqdiv_rvalue = IndR_his_uqvqdiv_regress["rvalue"]
IndR_his_uqvqdiv_pvalue = IndR_his_uqvqdiv_regress["pvalue"]

#### ssp585

In [14]:
nIndR_ssp585_p3_pre_slope = nIndR_ssp585_p3_pre_regress["slope"]
nIndR_ssp585_p3_pre_rvalue = nIndR_ssp585_p3_pre_regress["rvalue"]
nIndR_ssp585_p3_pre_pvalue = nIndR_ssp585_p3_pre_regress["pvalue"]

nIndR_ssp585_p3_hgt_slope = nIndR_ssp585_p3_hgt_regress["slope"]
nIndR_ssp585_p3_hgt_rvalue = nIndR_ssp585_p3_hgt_regress["rvalue"]
nIndR_ssp585_p3_hgt_pvalue = nIndR_ssp585_p3_hgt_regress["pvalue"]

nIndR_ssp585_p3_u_slope = nIndR_ssp585_p3_u_regress["slope"]
nIndR_ssp585_p3_u_rvalue = nIndR_ssp585_p3_u_regress["rvalue"]
nIndR_ssp585_p3_u_pvalue = nIndR_ssp585_p3_u_regress["pvalue"]

nIndR_ssp585_p3_v_slope = nIndR_ssp585_p3_v_regress["slope"]
nIndR_ssp585_p3_v_rvalue = nIndR_ssp585_p3_v_regress["rvalue"]
nIndR_ssp585_p3_v_pvalue = nIndR_ssp585_p3_v_regress["pvalue"]

# nIndR_ssp585_p3_wDJF_slope = nIndR_ssp585_p3_wDJF_regress["slope"]
# nIndR_ssp585_p3_wDJF_rvalue = nIndR_ssp585_p3_wDJF_regress["rvalue"]
# nIndR_ssp585_p3_wDJF_pvalue = nIndR_ssp585_p3_wDJF_regress["pvalue"]

# nIndR_ssp585_p3_wMAM_slope = nIndR_ssp585_p3_wMAM_regress["slope"]
# nIndR_ssp585_p3_wMAM_rvalue = nIndR_ssp585_p3_wMAM_regress["rvalue"]
# nIndR_ssp585_p3_wMAM_pvalue = nIndR_ssp585_p3_wMAM_regress["pvalue"]

# nIndR_ssp585_p3_wJJA_slope = nIndR_ssp585_p3_wJJA_regress["slope"]
# nIndR_ssp585_p3_wJJA_rvalue = nIndR_ssp585_p3_wJJA_regress["rvalue"]
# nIndR_ssp585_p3_wJJA_pvalue = nIndR_ssp585_p3_wJJA_regress["pvalue"]

# nIndR_ssp585_p3_wSON_slope = nIndR_ssp585_p3_wSON_regress["slope"]
# nIndR_ssp585_p3_wSON_rvalue = nIndR_ssp585_p3_wSON_regress["rvalue"]
# nIndR_ssp585_p3_wSON_pvalue = nIndR_ssp585_p3_wSON_regress["pvalue"]

# nIndR_ssp585_p3_wnDJF_slope = nIndR_ssp585_p3_wnDJF_regress["slope"]
# nIndR_ssp585_p3_wnDJF_rvalue = nIndR_ssp585_p3_wnDJF_regress["rvalue"]
# nIndR_ssp585_p3_wnDJF_pvalue = nIndR_ssp585_p3_wnDJF_regress["pvalue"]

# nIndR_ssp585_p3_div_slope = nIndR_ssp585_p3_div_regress["slope"]
# nIndR_ssp585_p3_div_rvalue = nIndR_ssp585_p3_div_regress["rvalue"]
# nIndR_ssp585_p3_div_pvalue = nIndR_ssp585_p3_div_regress["pvalue"]

nIndR_ssp585_p3_sstDJF_slope = nIndR_ssp585_p3_sstDJF_regress["slope"]
nIndR_ssp585_p3_sstDJF_rvalue = nIndR_ssp585_p3_sstDJF_regress["rvalue"]
nIndR_ssp585_p3_sstDJF_pvalue = nIndR_ssp585_p3_sstDJF_regress["pvalue"]

nIndR_ssp585_p3_sstMAM_slope = nIndR_ssp585_p3_sstMAM_regress["slope"]
nIndR_ssp585_p3_sstMAM_rvalue = nIndR_ssp585_p3_sstMAM_regress["rvalue"]
nIndR_ssp585_p3_sstMAM_pvalue = nIndR_ssp585_p3_sstMAM_regress["pvalue"]

nIndR_ssp585_p3_sstJJA_slope = nIndR_ssp585_p3_sstJJA_regress["slope"]
nIndR_ssp585_p3_sstJJA_rvalue = nIndR_ssp585_p3_sstJJA_regress["rvalue"]
nIndR_ssp585_p3_sstJJA_pvalue = nIndR_ssp585_p3_sstJJA_regress["pvalue"]

nIndR_ssp585_p3_sstSON_slope = nIndR_ssp585_p3_sstSON_regress["slope"]
nIndR_ssp585_p3_sstSON_rvalue = nIndR_ssp585_p3_sstSON_regress["rvalue"]
nIndR_ssp585_p3_sstSON_pvalue = nIndR_ssp585_p3_sstSON_regress["pvalue"]

nIndR_ssp585_p3_sstnDJF_slope = nIndR_ssp585_p3_sstnDJF_regress["slope"]
nIndR_ssp585_p3_sstnDJF_rvalue = nIndR_ssp585_p3_sstnDJF_regress["rvalue"]
nIndR_ssp585_p3_sstnDJF_pvalue = nIndR_ssp585_p3_sstnDJF_regress["pvalue"]

nIndR_ssp585_p3_uq_slope = nIndR_ssp585_p3_uq_regress["slope"]
nIndR_ssp585_p3_uq_rvalue = nIndR_ssp585_p3_uq_regress["rvalue"]
nIndR_ssp585_p3_uq_pvalue = nIndR_ssp585_p3_uq_regress["pvalue"]

nIndR_ssp585_p3_vq_slope = nIndR_ssp585_p3_vq_regress["slope"]
nIndR_ssp585_p3_vq_rvalue = nIndR_ssp585_p3_vq_regress["rvalue"]
nIndR_ssp585_p3_vq_pvalue = nIndR_ssp585_p3_vq_regress["pvalue"]

nIndR_ssp585_p3_uqvqdiv_slope = nIndR_ssp585_p3_uqvqdiv_regress["slope"]
nIndR_ssp585_p3_uqvqdiv_rvalue = nIndR_ssp585_p3_uqvqdiv_regress["rvalue"]
nIndR_ssp585_p3_uqvqdiv_pvalue = nIndR_ssp585_p3_uqvqdiv_regress["pvalue"]

In [15]:
sIndR_ssp585_p3_pre_slope = sIndR_ssp585_p3_pre_regress["slope"]
sIndR_ssp585_p3_pre_rvalue = sIndR_ssp585_p3_pre_regress["rvalue"]
sIndR_ssp585_p3_pre_pvalue = sIndR_ssp585_p3_pre_regress["pvalue"]

sIndR_ssp585_p3_hgt_slope = sIndR_ssp585_p3_hgt_regress["slope"]
sIndR_ssp585_p3_hgt_rvalue = sIndR_ssp585_p3_hgt_regress["rvalue"]
sIndR_ssp585_p3_hgt_pvalue = sIndR_ssp585_p3_hgt_regress["pvalue"]

sIndR_ssp585_p3_u_slope = sIndR_ssp585_p3_u_regress["slope"]
sIndR_ssp585_p3_u_rvalue = sIndR_ssp585_p3_u_regress["rvalue"]
sIndR_ssp585_p3_u_pvalue = sIndR_ssp585_p3_u_regress["pvalue"]

sIndR_ssp585_p3_v_slope = sIndR_ssp585_p3_v_regress["slope"]
sIndR_ssp585_p3_v_rvalue = sIndR_ssp585_p3_v_regress["rvalue"]
sIndR_ssp585_p3_v_pvalue = sIndR_ssp585_p3_v_regress["pvalue"]

# sIndR_ssp585_p3_wDJF_slope = sIndR_ssp585_p3_wDJF_regress["slope"]
# sIndR_ssp585_p3_wDJF_rvalue = sIndR_ssp585_p3_wDJF_regress["rvalue"]
# sIndR_ssp585_p3_wDJF_pvalue = sIndR_ssp585_p3_wDJF_regress["pvalue"]

# sIndR_ssp585_p3_wMAM_slope = sIndR_ssp585_p3_wMAM_regress["slope"]
# sIndR_ssp585_p3_wMAM_rvalue = sIndR_ssp585_p3_wMAM_regress["rvalue"]
# sIndR_ssp585_p3_wMAM_pvalue = sIndR_ssp585_p3_wMAM_regress["pvalue"]

# sIndR_ssp585_p3_wJJA_slope = sIndR_ssp585_p3_wJJA_regress["slope"]
# sIndR_ssp585_p3_wJJA_rvalue = sIndR_ssp585_p3_wJJA_regress["rvalue"]
# sIndR_ssp585_p3_wJJA_pvalue = sIndR_ssp585_p3_wJJA_regress["pvalue"]

# sIndR_ssp585_p3_wSON_slope = sIndR_ssp585_p3_wSON_regress["slope"]
# sIndR_ssp585_p3_wSON_rvalue = sIndR_ssp585_p3_wSON_regress["rvalue"]
# sIndR_ssp585_p3_wSON_pvalue = sIndR_ssp585_p3_wSON_regress["pvalue"]

# sIndR_ssp585_p3_wnDJF_slope = sIndR_ssp585_p3_wnDJF_regress["slope"]
# sIndR_ssp585_p3_wnDJF_rvalue = sIndR_ssp585_p3_wnDJF_regress["rvalue"]
# sIndR_ssp585_p3_wnDJF_pvalue = sIndR_ssp585_p3_wnDJF_regress["pvalue"]

# sIndR_ssp585_p3_div_slope = sIndR_ssp585_p3_div_regress["slope"]
# sIndR_ssp585_p3_div_rvalue = sIndR_ssp585_p3_div_regress["rvalue"]
# sIndR_ssp585_p3_div_pvalue = sIndR_ssp585_p3_div_regress["pvalue"]

sIndR_ssp585_p3_sstDJF_slope = sIndR_ssp585_p3_sstDJF_regress["slope"]
sIndR_ssp585_p3_sstDJF_rvalue = sIndR_ssp585_p3_sstDJF_regress["rvalue"]
sIndR_ssp585_p3_sstDJF_pvalue = sIndR_ssp585_p3_sstDJF_regress["pvalue"]

sIndR_ssp585_p3_sstMAM_slope = sIndR_ssp585_p3_sstMAM_regress["slope"]
sIndR_ssp585_p3_sstMAM_rvalue = sIndR_ssp585_p3_sstMAM_regress["rvalue"]
sIndR_ssp585_p3_sstMAM_pvalue = sIndR_ssp585_p3_sstMAM_regress["pvalue"]

sIndR_ssp585_p3_sstJJA_slope = sIndR_ssp585_p3_sstJJA_regress["slope"]
sIndR_ssp585_p3_sstJJA_rvalue = sIndR_ssp585_p3_sstJJA_regress["rvalue"]
sIndR_ssp585_p3_sstJJA_pvalue = sIndR_ssp585_p3_sstJJA_regress["pvalue"]

sIndR_ssp585_p3_sstSON_slope = sIndR_ssp585_p3_sstSON_regress["slope"]
sIndR_ssp585_p3_sstSON_rvalue = sIndR_ssp585_p3_sstSON_regress["rvalue"]
sIndR_ssp585_p3_sstSON_pvalue = sIndR_ssp585_p3_sstSON_regress["pvalue"]

sIndR_ssp585_p3_sstnDJF_slope = sIndR_ssp585_p3_sstnDJF_regress["slope"]
sIndR_ssp585_p3_sstnDJF_rvalue = sIndR_ssp585_p3_sstnDJF_regress["rvalue"]
sIndR_ssp585_p3_sstnDJF_pvalue = sIndR_ssp585_p3_sstnDJF_regress["pvalue"]

sIndR_ssp585_p3_uq_slope = sIndR_ssp585_p3_uq_regress["slope"]
sIndR_ssp585_p3_uq_rvalue = sIndR_ssp585_p3_uq_regress["rvalue"]
sIndR_ssp585_p3_uq_pvalue = sIndR_ssp585_p3_uq_regress["pvalue"]

sIndR_ssp585_p3_vq_slope = sIndR_ssp585_p3_vq_regress["slope"]
sIndR_ssp585_p3_vq_rvalue = sIndR_ssp585_p3_vq_regress["rvalue"]
sIndR_ssp585_p3_vq_pvalue = sIndR_ssp585_p3_vq_regress["pvalue"]

sIndR_ssp585_p3_uqvqdiv_slope = sIndR_ssp585_p3_uqvqdiv_regress["slope"]
sIndR_ssp585_p3_uqvqdiv_rvalue = sIndR_ssp585_p3_uqvqdiv_regress["rvalue"]
sIndR_ssp585_p3_uqvqdiv_pvalue = sIndR_ssp585_p3_uqvqdiv_regress["pvalue"]

In [16]:
IndR_ssp585_p3_pre_slope = IndR_ssp585_p3_pre_regress["slope"]
IndR_ssp585_p3_pre_rvalue = IndR_ssp585_p3_pre_regress["rvalue"]
IndR_ssp585_p3_pre_pvalue = IndR_ssp585_p3_pre_regress["pvalue"]

IndR_ssp585_p3_hgt_slope = IndR_ssp585_p3_hgt_regress["slope"]
IndR_ssp585_p3_hgt_rvalue = IndR_ssp585_p3_hgt_regress["rvalue"]
IndR_ssp585_p3_hgt_pvalue = IndR_ssp585_p3_hgt_regress["pvalue"]

IndR_ssp585_p3_u_slope = IndR_ssp585_p3_u_regress["slope"]
IndR_ssp585_p3_u_rvalue = IndR_ssp585_p3_u_regress["rvalue"]
IndR_ssp585_p3_u_pvalue = IndR_ssp585_p3_u_regress["pvalue"]

IndR_ssp585_p3_v_slope = IndR_ssp585_p3_v_regress["slope"]
IndR_ssp585_p3_v_rvalue = IndR_ssp585_p3_v_regress["rvalue"]
IndR_ssp585_p3_v_pvalue = IndR_ssp585_p3_v_regress["pvalue"]

# IndR_ssp585_p3_wDJF_slope = IndR_ssp585_p3_wDJF_regress["slope"]
# IndR_ssp585_p3_wDJF_rvalue = IndR_ssp585_p3_wDJF_regress["rvalue"]
# IndR_ssp585_p3_wDJF_pvalue = IndR_ssp585_p3_wDJF_regress["pvalue"]

# IndR_ssp585_p3_wMAM_slope = IndR_ssp585_p3_wMAM_regress["slope"]
# IndR_ssp585_p3_wMAM_rvalue = IndR_ssp585_p3_wMAM_regress["rvalue"]
# IndR_ssp585_p3_wMAM_pvalue = IndR_ssp585_p3_wMAM_regress["pvalue"]

# IndR_ssp585_p3_wJJA_slope = IndR_ssp585_p3_wJJA_regress["slope"]
# IndR_ssp585_p3_wJJA_rvalue = IndR_ssp585_p3_wJJA_regress["rvalue"]
# IndR_ssp585_p3_wJJA_pvalue = IndR_ssp585_p3_wJJA_regress["pvalue"]

# IndR_ssp585_p3_wSON_slope = IndR_ssp585_p3_wSON_regress["slope"]
# IndR_ssp585_p3_wSON_rvalue = IndR_ssp585_p3_wSON_regress["rvalue"]
# IndR_ssp585_p3_wSON_pvalue = IndR_ssp585_p3_wSON_regress["pvalue"]

# IndR_ssp585_p3_wnDJF_slope = IndR_ssp585_p3_wnDJF_regress["slope"]
# IndR_ssp585_p3_wnDJF_rvalue = IndR_ssp585_p3_wnDJF_regress["rvalue"]
# IndR_ssp585_p3_wnDJF_pvalue = IndR_ssp585_p3_wnDJF_regress["pvalue"]

# IndR_ssp585_p3_div_slope = IndR_ssp585_p3_div_regress["slope"]
# IndR_ssp585_p3_div_rvalue = IndR_ssp585_p3_div_regress["rvalue"]
# IndR_ssp585_p3_div_pvalue = IndR_ssp585_p3_div_regress["pvalue"]

IndR_ssp585_p3_sstDJF_slope = IndR_ssp585_p3_sstDJF_regress["slope"]
IndR_ssp585_p3_sstDJF_rvalue = IndR_ssp585_p3_sstDJF_regress["rvalue"]
IndR_ssp585_p3_sstDJF_pvalue = IndR_ssp585_p3_sstDJF_regress["pvalue"]

IndR_ssp585_p3_sstMAM_slope = IndR_ssp585_p3_sstMAM_regress["slope"]
IndR_ssp585_p3_sstMAM_rvalue = IndR_ssp585_p3_sstMAM_regress["rvalue"]
IndR_ssp585_p3_sstMAM_pvalue = IndR_ssp585_p3_sstMAM_regress["pvalue"]

IndR_ssp585_p3_sstJJA_slope = IndR_ssp585_p3_sstJJA_regress["slope"]
IndR_ssp585_p3_sstJJA_rvalue = IndR_ssp585_p3_sstJJA_regress["rvalue"]
IndR_ssp585_p3_sstJJA_pvalue = IndR_ssp585_p3_sstJJA_regress["pvalue"]

IndR_ssp585_p3_sstSON_slope = IndR_ssp585_p3_sstSON_regress["slope"]
IndR_ssp585_p3_sstSON_rvalue = IndR_ssp585_p3_sstSON_regress["rvalue"]
IndR_ssp585_p3_sstSON_pvalue = IndR_ssp585_p3_sstSON_regress["pvalue"]

IndR_ssp585_p3_sstnDJF_slope = IndR_ssp585_p3_sstnDJF_regress["slope"]
IndR_ssp585_p3_sstnDJF_rvalue = IndR_ssp585_p3_sstnDJF_regress["rvalue"]
IndR_ssp585_p3_sstnDJF_pvalue = IndR_ssp585_p3_sstnDJF_regress["pvalue"]

IndR_ssp585_p3_uq_slope = IndR_ssp585_p3_uq_regress["slope"]
IndR_ssp585_p3_uq_rvalue = IndR_ssp585_p3_uq_regress["rvalue"]
IndR_ssp585_p3_uq_pvalue = IndR_ssp585_p3_uq_regress["pvalue"]

IndR_ssp585_p3_vq_slope = IndR_ssp585_p3_vq_regress["slope"]
IndR_ssp585_p3_vq_rvalue = IndR_ssp585_p3_vq_regress["rvalue"]
IndR_ssp585_p3_vq_pvalue = IndR_ssp585_p3_vq_regress["pvalue"]

IndR_ssp585_p3_uqvqdiv_slope = IndR_ssp585_p3_uqvqdiv_regress["slope"]
IndR_ssp585_p3_uqvqdiv_rvalue = IndR_ssp585_p3_uqvqdiv_regress["rvalue"]
IndR_ssp585_p3_uqvqdiv_pvalue = IndR_ssp585_p3_uqvqdiv_regress["pvalue"]

#### difference

In [17]:
IndR_diff_pre_slope = IndR_ssp585_p3_pre_slope - IndR_his_pre_slope
IndR_diff_hgt_slope = IndR_ssp585_p3_hgt_slope - IndR_his_hgt_slope
IndR_diff_u_slope = IndR_ssp585_p3_u_slope - IndR_his_u_slope
IndR_diff_v_slope = IndR_ssp585_p3_v_slope - IndR_his_v_slope
# IndR_diff_wDJF_slope = IndR_ssp585_p3_wDJF_slope - IndR_his_wDJF_slope
# IndR_diff_wMAM_slope = IndR_ssp585_p3_wMAM_slope - IndR_his_wMAM_slope
# IndR_diff_wJJA_slope = IndR_ssp585_p3_wJJA_slope - IndR_his_wJJA_slope
# IndR_diff_wSON_slope = IndR_ssp585_p3_wSON_slope - IndR_his_wSON_slope
# IndR_diff_wnDJF_slope = IndR_ssp585_p3_wnDJF_slope - IndR_his_wnDJF_slope
# IndR_diff_div_slope = IndR_ssp585_p3_div_slope - IndR_his_div_slope

IndR_diff_sstDJF_slope = IndR_ssp585_p3_sstDJF_slope - IndR_his_sstDJF_slope
IndR_diff_sstMAM_slope = IndR_ssp585_p3_sstMAM_slope - IndR_his_sstMAM_slope
IndR_diff_sstJJA_slope = IndR_ssp585_p3_sstJJA_slope - IndR_his_sstJJA_slope
IndR_diff_sstSON_slope = IndR_ssp585_p3_sstSON_slope - IndR_his_sstSON_slope
IndR_diff_sstnDJF_slope = IndR_ssp585_p3_sstnDJF_slope - IndR_his_sstnDJF_slope

IndR_diff_uq_slope = IndR_ssp585_p3_uq_slope - IndR_his_uq_slope
IndR_diff_vq_slope = IndR_ssp585_p3_vq_slope - IndR_his_vq_slope
IndR_diff_uqvqdiv_slope = IndR_ssp585_p3_uqvqdiv_slope - IndR_his_uqvqdiv_slope
#===============================================================================
nIndR_diff_pre_slope = nIndR_ssp585_p3_pre_slope - nIndR_his_pre_slope
nIndR_diff_hgt_slope = nIndR_ssp585_p3_hgt_slope - nIndR_his_hgt_slope
nIndR_diff_u_slope = nIndR_ssp585_p3_u_slope - nIndR_his_u_slope
nIndR_diff_v_slope = nIndR_ssp585_p3_v_slope - nIndR_his_v_slope
# nIndR_diff_wDJF_slope = nIndR_ssp585_p3_wDJF_slope - nIndR_his_wDJF_slope
# nIndR_diff_wMAM_slope = nIndR_ssp585_p3_wMAM_slope - nIndR_his_wMAM_slope
# nIndR_diff_wJJA_slope = nIndR_ssp585_p3_wJJA_slope - nIndR_his_wJJA_slope
# nIndR_diff_wSON_slope = nIndR_ssp585_p3_wSON_slope - nIndR_his_wSON_slope
# nIndR_diff_wnDJF_slope = nIndR_ssp585_p3_wnDJF_slope - nIndR_his_wnDJF_slope
# nIndR_diff_div_slope = nIndR_ssp585_p3_div_slope - nIndR_his_div_slope

nIndR_diff_sstDJF_slope = nIndR_ssp585_p3_sstDJF_slope - nIndR_his_sstDJF_slope
nIndR_diff_sstMAM_slope = nIndR_ssp585_p3_sstMAM_slope - nIndR_his_sstMAM_slope
nIndR_diff_sstJJA_slope = nIndR_ssp585_p3_sstJJA_slope - nIndR_his_sstJJA_slope
nIndR_diff_sstSON_slope = nIndR_ssp585_p3_sstSON_slope - nIndR_his_sstSON_slope
nIndR_diff_sstnDJF_slope = nIndR_ssp585_p3_sstnDJF_slope - nIndR_his_sstnDJF_slope

nIndR_diff_uq_slope = nIndR_ssp585_p3_uq_slope - nIndR_his_uq_slope
nIndR_diff_vq_slope = nIndR_ssp585_p3_vq_slope - nIndR_his_vq_slope
nIndR_diff_uqvqdiv_slope = nIndR_ssp585_p3_uqvqdiv_slope - nIndR_his_uqvqdiv_slope
#===============================================================================
sIndR_diff_pre_slope = sIndR_ssp585_p3_pre_slope - sIndR_his_pre_slope
sIndR_diff_hgt_slope = sIndR_ssp585_p3_hgt_slope - sIndR_his_hgt_slope
sIndR_diff_u_slope = sIndR_ssp585_p3_u_slope - sIndR_his_u_slope
sIndR_diff_v_slope = sIndR_ssp585_p3_v_slope - sIndR_his_v_slope
# sIndR_diff_wDJF_slope = sIndR_ssp585_p3_wDJF_slope - sIndR_his_wDJF_slope
# sIndR_diff_wMAM_slope = sIndR_ssp585_p3_wMAM_slope - sIndR_his_wMAM_slope
# sIndR_diff_wJJA_slope = sIndR_ssp585_p3_wJJA_slope - sIndR_his_wJJA_slope
# sIndR_diff_wSON_slope = sIndR_ssp585_p3_wSON_slope - sIndR_his_wSON_slope
# sIndR_diff_wnDJF_slope = sIndR_ssp585_p3_wnDJF_slope - sIndR_his_wnDJF_slope
# sIndR_diff_div_slope = sIndR_ssp585_p3_div_slope - sIndR_his_div_slope

sIndR_diff_sstDJF_slope = sIndR_ssp585_p3_sstDJF_slope - sIndR_his_sstDJF_slope
sIndR_diff_sstMAM_slope = sIndR_ssp585_p3_sstMAM_slope - sIndR_his_sstMAM_slope
sIndR_diff_sstJJA_slope = sIndR_ssp585_p3_sstJJA_slope - sIndR_his_sstJJA_slope
sIndR_diff_sstSON_slope = sIndR_ssp585_p3_sstSON_slope - sIndR_his_sstSON_slope
sIndR_diff_sstnDJF_slope = sIndR_ssp585_p3_sstnDJF_slope - sIndR_his_sstnDJF_slope

sIndR_diff_uq_slope = sIndR_ssp585_p3_uq_slope - sIndR_his_uq_slope
sIndR_diff_vq_slope = sIndR_ssp585_p3_vq_slope - sIndR_his_vq_slope
sIndR_diff_uqvqdiv_slope = sIndR_ssp585_p3_uqvqdiv_slope - sIndR_his_uqvqdiv_slope

## 计算高层和低层的好模式

In [18]:
lon_ranking1_W = 50
lon_ranking1_E = 137.5
lat_ranking1_N = 47.5
lat_ranking1_S = 15.0

lat = IndR_his_hgt_slope.lat
lon= IndR_his_hgt_slope.lon

#   for 200hPa
lat_ranking_range1 = lat[(lat>=lat_ranking1_S) & (lat<=lat_ranking1_N)]
lon_ranking_range1 = lon[(lon>=lon_ranking1_W) & (lon<=lon_ranking1_E)]
IndR_200hgt_pcc = []
IndR_200hgt_RMSE = []
IndR_200hgt_std = []

for num_mod, mod in enumerate(models):
    
    hgt200_pcc = ca.cal_pcc(nIndR_ERA5_hgt_slope[:,:,:].sel(lat=lat_ranking_range1, lon=lon_ranking_range1, level=200.0), nIndR_his_hgt_slope[:,:,:,:].sel(models=mod, lat=lat_ranking_range1, lon=lon_ranking_range1, level=200.0))
    
    IndR_200hgt_pcc.append(hgt200_pcc)

    IndR_200hgt_RMSE.append(np.sqrt(np.power((nIndR_his_hgt_slope[:,:,:,:].sel(models=mod,lat=lat_ranking_range1,lon=lon_ranking_range1, level=200.0)-nIndR_ERA5_hgt_slope[:,:,:].sel(lat=lat_ranking_range1, lon=lon_ranking_range1, level=200.0)),2).mean(dim=["lat","lon"],skipna=True).data))
    
    IndR_200hgt_std.append(float((nIndR_his_hgt_slope[:,:,:,:].sel(models=mod,lat=lat_ranking_range1,lon=lon_ranking_range1, level=200.0).std(dim=["lat","lon"],skipna=True)/nIndR_ERA5_hgt_slope[:,:,:].sel(lat=lat_ranking_range1, lon=lon_ranking_range1, level=200.0).std(dim=["lat","lon"],skipna=True)).data))


In [22]:
models.where(np.array(IndR_200hgt_pcc)>=0.90)

<xarray.DataArray 'models' (models: 26)>
array([nan, nan, 'CAMS-CSM1-0', nan, nan, 'CESM2-WACCM', nan, nan, nan,
       'EC-Earth3', nan, nan, nan, 'HadGEM3-GC31-LL', nan, nan, nan, nan,
       nan, nan, 'MPI-ESM1-2-HR', nan, 'NESM3', nan, nan, 'UKESM1-0-LL'],
      dtype=object)
Coordinates:
  * models   (models) object 'ACCESS-CM2' 'BCC-CSM2-MR' ... 'UKESM1-0-LL'

## 集合平均

### historical

In [ ]:
nIndR_his_hgt_slope_ens = nIndR_his_hgt_slope.mean(dim="models",skipna=True)
sIndR_his_hgt_slope_ens = sIndR_his_hgt_slope.mean(dim="models",skipna=True)
IndR_his_hgt_slope_ens = IndR_his_hgt_slope.mean(dim="models",skipna=True)

nIndR_his_u_slope_ens = nIndR_his_u_slope.mean(dim="models",skipna=True)
sIndR_his_u_slope_ens = sIndR_his_u_slope.mean(dim="models",skipna=True)
IndR_his_u_slope_ens = IndR_his_u_slope.mean(dim="models",skipna=True)

nIndR_his_v_slope_ens = nIndR_his_v_slope.mean(dim="models",skipna=True)
sIndR_his_v_slope_ens = sIndR_his_v_slope.mean(dim="models",skipna=True)
IndR_his_v_slope_ens = IndR_his_v_slope.mean(dim="models",skipna=True)

nIndR_his_pre_slope_ens = nIndR_his_pre_slope.mean(dim="models",skipna=True)
sIndR_his_pre_slope_ens = sIndR_his_pre_slope.mean(dim="models",skipna=True)
IndR_his_pre_slope_ens = IndR_his_pre_slope.mean(dim="models",skipna=True)

nIndR_his_uq_slope_ens = nIndR_his_uq_slope.mean(dim="models",skipna=True)
sIndR_his_uq_slope_ens = sIndR_his_uq_slope.mean(dim="models",skipna=True)
IndR_his_uq_slope_ens = IndR_his_uq_slope.mean(dim="models",skipna=True)

nIndR_his_vq_slope_ens = nIndR_his_vq_slope.mean(dim="models",skipna=True)
sIndR_his_vq_slope_ens = sIndR_his_vq_slope.mean(dim="models",skipna=True)
IndR_his_vq_slope_ens = IndR_his_vq_slope.mean(dim="models",skipna=True)

nIndR_his_uqvqdiv_slope_ens = nIndR_his_uqvqdiv_slope.mean(dim="models",skipna=True)
sIndR_his_uqvqdiv_slope_ens = sIndR_his_uqvqdiv_slope.mean(dim="models",skipna=True)
IndR_his_uqvqdiv_slope_ens = IndR_his_uqvqdiv_slope.mean(dim="models",skipna=True)

# nIndR_his_div_slope_ens = nIndR_his_div_slope.mean(dim="models",skipna=True)
# sIndR_his_div_slope_ens = sIndR_his_div_slope.mean(dim="models",skipna=True)
# IndR_his_div_slope_ens = IndR_his_div_slope.mean(dim="models",skipna=True)

nIndR_his_sstDJF_slope_ens = nIndR_his_sstDJF_slope.mean(dim="models",skipna=True)
sIndR_his_sstDJF_slope_ens = sIndR_his_sstDJF_slope.mean(dim="models",skipna=True)
IndR_his_sstDJF_slope_ens = IndR_his_sstDJF_slope.mean(dim="models",skipna=True)

nIndR_his_sstMAM_slope_ens = nIndR_his_sstMAM_slope.mean(dim="models",skipna=True)
sIndR_his_sstMAM_slope_ens = sIndR_his_sstMAM_slope.mean(dim="models",skipna=True)
IndR_his_sstMAM_slope_ens = IndR_his_sstMAM_slope.mean(dim="models",skipna=True)

nIndR_his_sstJJA_slope_ens = nIndR_his_sstJJA_slope.mean(dim="models",skipna=True)
sIndR_his_sstJJA_slope_ens = sIndR_his_sstJJA_slope.mean(dim="models",skipna=True)
IndR_his_sstJJA_slope_ens = IndR_his_sstJJA_slope.mean(dim="models",skipna=True)

nIndR_his_sstSON_slope_ens = nIndR_his_sstSON_slope.mean(dim="models",skipna=True)
sIndR_his_sstSON_slope_ens = sIndR_his_sstSON_slope.mean(dim="models",skipna=True)
IndR_his_sstSON_slope_ens = IndR_his_sstSON_slope.mean(dim="models",skipna=True)

nIndR_his_sstnDJF_slope_ens = nIndR_his_sstnDJF_slope.mean(dim="models",skipna=True)
sIndR_his_sstnDJF_slope_ens = sIndR_his_sstnDJF_slope.mean(dim="models",skipna=True)
IndR_his_sstnDJF_slope_ens = IndR_his_sstnDJF_slope.mean(dim="models",skipna=True)

### ssp585_p3

In [ ]:
nIndR_ssp585_p3_hgt_slope_ens = nIndR_ssp585_p3_hgt_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_hgt_slope_ens = sIndR_ssp585_p3_hgt_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_hgt_slope_ens = IndR_ssp585_p3_hgt_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_u_slope_ens = nIndR_ssp585_p3_u_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_u_slope_ens = sIndR_ssp585_p3_u_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_u_slope_ens = IndR_ssp585_p3_u_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_v_slope_ens = nIndR_ssp585_p3_v_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_v_slope_ens = sIndR_ssp585_p3_v_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_v_slope_ens = IndR_ssp585_p3_v_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_pre_slope_ens = nIndR_ssp585_p3_pre_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_pre_slope_ens = sIndR_ssp585_p3_pre_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_pre_slope_ens = IndR_ssp585_p3_pre_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_uq_slope_ens = nIndR_ssp585_p3_uq_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_uq_slope_ens = sIndR_ssp585_p3_uq_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_uq_slope_ens = IndR_ssp585_p3_uq_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_vq_slope_ens = nIndR_ssp585_p3_vq_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_vq_slope_ens = sIndR_ssp585_p3_vq_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_vq_slope_ens = IndR_ssp585_p3_vq_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_uqvqdiv_slope_ens = nIndR_ssp585_p3_uqvqdiv_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_uqvqdiv_slope_ens = sIndR_ssp585_p3_uqvqdiv_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_uqvqdiv_slope_ens = IndR_ssp585_p3_uqvqdiv_slope.mean(dim="models",skipna=True)

# nIndR_ssp585_p3_div_slope_ens = nIndR_ssp585_p3_div_slope.mean(dim="models",skipna=True)
# sIndR_ssp585_p3_div_slope_ens = sIndR_ssp585_p3_div_slope.mean(dim="models",skipna=True)
# IndR_ssp585_p3_div_slope_ens = IndR_ssp585_p3_div_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_sstDJF_slope_ens = nIndR_ssp585_p3_sstDJF_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_sstDJF_slope_ens = sIndR_ssp585_p3_sstDJF_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_sstDJF_slope_ens = IndR_ssp585_p3_sstDJF_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_sstMAM_slope_ens = nIndR_ssp585_p3_sstMAM_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_sstMAM_slope_ens = sIndR_ssp585_p3_sstMAM_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_sstMAM_slope_ens = IndR_ssp585_p3_sstMAM_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_sstJJA_slope_ens = nIndR_ssp585_p3_sstJJA_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_sstJJA_slope_ens = sIndR_ssp585_p3_sstJJA_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_sstJJA_slope_ens = IndR_ssp585_p3_sstJJA_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_sstSON_slope_ens = nIndR_ssp585_p3_sstSON_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_sstSON_slope_ens = sIndR_ssp585_p3_sstSON_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_sstSON_slope_ens = IndR_ssp585_p3_sstSON_slope.mean(dim="models",skipna=True)

nIndR_ssp585_p3_sstnDJF_slope_ens = nIndR_ssp585_p3_sstnDJF_slope.mean(dim="models",skipna=True)
sIndR_ssp585_p3_sstnDJF_slope_ens = sIndR_ssp585_p3_sstnDJF_slope.mean(dim="models",skipna=True)
IndR_ssp585_p3_sstnDJF_slope_ens = IndR_ssp585_p3_sstnDJF_slope.mean(dim="models",skipna=True)

### 计算difference的集合平均

In [ ]:
IndR_diff_pre_slope_ens = IndR_diff_pre_slope.mean(dim="models",skipna=True)
IndR_diff_hgt_slope_ens = IndR_diff_hgt_slope.mean(dim="models",skipna=True)
IndR_diff_u_slope_ens = IndR_diff_u_slope.mean(dim="models",skipna=True)
IndR_diff_v_slope_ens = IndR_diff_v_slope.mean(dim="models",skipna=True)
IndR_diff_sstDJF_slope_ens = IndR_diff_sstDJF_slope.mean(dim="models",skipna=True)
IndR_diff_sstMAM_slope_ens = IndR_diff_sstMAM_slope.mean(dim="models",skipna=True)
IndR_diff_sstJJA_slope_ens = IndR_diff_sstJJA_slope.mean(dim="models",skipna=True)
IndR_diff_sstSON_slope_ens = IndR_diff_sstSON_slope.mean(dim="models",skipna=True)
IndR_diff_sstnDJF_slope_ens = IndR_diff_sstnDJF_slope.mean(dim="models",skipna=True)
IndR_diff_uq_slope_ens = IndR_diff_uq_slope.mean(dim="models",skipna=True)
IndR_diff_vq_slope_ens = IndR_diff_vq_slope.mean(dim="models",skipna=True)
IndR_diff_uqvqdiv_slope_ens = IndR_diff_uqvqdiv_slope.mean(dim="models",skipna=True)

In [ ]:
nIndR_diff_pre_slope_ens = nIndR_diff_pre_slope.mean(dim="models",skipna=True)
nIndR_diff_hgt_slope_ens = nIndR_diff_hgt_slope.mean(dim="models",skipna=True)
nIndR_diff_u_slope_ens = nIndR_diff_u_slope.mean(dim="models",skipna=True)
nIndR_diff_v_slope_ens = nIndR_diff_v_slope.mean(dim="models",skipna=True)
nIndR_diff_sstDJF_slope_ens = nIndR_diff_sstDJF_slope.mean(dim="models",skipna=True)
nIndR_diff_sstMAM_slope_ens = nIndR_diff_sstMAM_slope.mean(dim="models",skipna=True)
nIndR_diff_sstJJA_slope_ens = nIndR_diff_sstJJA_slope.mean(dim="models",skipna=True)
nIndR_diff_sstSON_slope_ens = nIndR_diff_sstSON_slope.mean(dim="models",skipna=True)
nIndR_diff_sstnDJF_slope_ens = nIndR_diff_sstnDJF_slope.mean(dim="models",skipna=True)
nIndR_diff_uq_slope_ens = nIndR_diff_uq_slope.mean(dim="models",skipna=True)
nIndR_diff_vq_slope_ens = nIndR_diff_vq_slope.mean(dim="models",skipna=True)
nIndR_diff_uqvqdiv_slope_ens = nIndR_diff_uqvqdiv_slope.mean(dim="models",skipna=True)

In [ ]:
sIndR_diff_pre_slope_ens = sIndR_diff_pre_slope.mean(dim="models",skipna=True)
sIndR_diff_hgt_slope_ens = sIndR_diff_hgt_slope.mean(dim="models",skipna=True)
sIndR_diff_u_slope_ens = sIndR_diff_u_slope.mean(dim="models",skipna=True)
sIndR_diff_v_slope_ens = sIndR_diff_v_slope.mean(dim="models",skipna=True)
sIndR_diff_sstDJF_slope_ens = sIndR_diff_sstDJF_slope.mean(dim="models",skipna=True)
sIndR_diff_sstMAM_slope_ens = sIndR_diff_sstMAM_slope.mean(dim="models",skipna=True)
sIndR_diff_sstJJA_slope_ens = sIndR_diff_sstJJA_slope.mean(dim="models",skipna=True)
sIndR_diff_sstSON_slope_ens = sIndR_diff_sstSON_slope.mean(dim="models",skipna=True)
sIndR_diff_sstnDJF_slope_ens = sIndR_diff_sstnDJF_slope.mean(dim="models",skipna=True)
sIndR_diff_uq_slope_ens = sIndR_diff_uq_slope.mean(dim="models",skipna=True)
sIndR_diff_vq_slope_ens = sIndR_diff_vq_slope.mean(dim="models",skipna=True)
sIndR_diff_uqvqdiv_slope_ens = sIndR_diff_uqvqdiv_slope.mean(dim="models",skipna=True)

## 显著性检验

#### 观测

In [ ]:
nIndR_ERA5_wind_mask = ca.wind_check(
    xr.where(nIndR_ERA5_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ERA5_v_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ERA5_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ERA5_v_pvalue <= 0.05, 1.0, 0.0),
)
sIndR_ERA5_wind_mask = ca.wind_check(
    xr.where(sIndR_ERA5_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ERA5_v_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ERA5_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ERA5_v_pvalue <= 0.05, 1.0, 0.0),
)
IndR_ERA5_wind_mask = ca.wind_check(
    xr.where(IndR_ERA5_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ERA5_v_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ERA5_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ERA5_v_pvalue <= 0.05, 1.0, 0.0),
)

nIndR_ERA5_uqvq_mask = ca.wind_check(
    xr.where(nIndR_ERA5_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ERA5_vq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ERA5_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ERA5_vq_pvalue <= 0.05, 1.0, 0.0),
)
sIndR_ERA5_uqvq_mask = ca.wind_check(
    xr.where(sIndR_ERA5_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ERA5_vq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ERA5_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ERA5_vq_pvalue <= 0.05, 1.0, 0.0),
)
IndR_ERA5_uqvq_mask = ca.wind_check(
    xr.where(IndR_ERA5_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ERA5_vq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ERA5_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ERA5_vq_pvalue <= 0.05, 1.0, 0.0),
)

### historical

In [ ]:
nIndR_his_wind_mask = ca.wind_check(
    xr.where(nIndR_his_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_his_v_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_his_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_his_v_pvalue <= 0.05, 1.0, 0.0),
)
sIndR_his_wind_mask = ca.wind_check(
    xr.where(sIndR_his_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_his_v_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_his_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_his_v_pvalue <= 0.05, 1.0, 0.0),
)
IndR_his_wind_mask = ca.wind_check(
    xr.where(IndR_his_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_his_v_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_his_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_his_v_pvalue <= 0.05, 1.0, 0.0),
)

nIndR_his_uqvq_mask = ca.wind_check(
    xr.where(nIndR_his_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_his_vq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_his_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_his_vq_pvalue <= 0.05, 1.0, 0.0),
)
sIndR_his_uqvq_mask = ca.wind_check(
    xr.where(sIndR_his_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_his_vq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_his_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_his_vq_pvalue <= 0.05, 1.0, 0.0),
)
IndR_his_uqvq_mask = ca.wind_check(
    xr.where(IndR_his_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_his_vq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_his_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_his_vq_pvalue <= 0.05, 1.0, 0.0),
)

### historical集合平均

In [ ]:
nIndR_his_hgt_slope_ens_mask = ca.MME_mask(nIndR_his_hgt_slope, big=True)
sIndR_his_hgt_slope_ens_mask = ca.MME_mask(sIndR_his_hgt_slope, big=True)
IndR_his_hgt_slope_ens_mask = ca.MME_mask(IndR_his_hgt_slope, big=True)

nIndR_his_u_slope_ens_mask = ca.MME_mask(nIndR_his_u_slope, big=True)
sIndR_his_u_slope_ens_mask = ca.MME_mask(sIndR_his_u_slope, big=True)
IndR_his_u_slope_ens_mask = ca.MME_mask(IndR_his_u_slope, big=True)

nIndR_his_v_slope_ens_mask = ca.MME_mask(nIndR_his_v_slope, big=True)
sIndR_his_v_slope_ens_mask = ca.MME_mask(sIndR_his_v_slope, big=True)
IndR_his_v_slope_ens_mask = ca.MME_mask(IndR_his_v_slope, big=True)

nIndR_his_pre_slope_ens_mask = ca.MME_mask(nIndR_his_pre_slope, big=True)
sIndR_his_pre_slope_ens_mask = ca.MME_mask(sIndR_his_pre_slope, big=True)
IndR_his_pre_slope_ens_mask = ca.MME_mask(IndR_his_pre_slope, big=True)

nIndR_his_uq_slope_ens_mask = ca.MME_mask(nIndR_his_uq_slope, big=True)
sIndR_his_uq_slope_ens_mask = ca.MME_mask(sIndR_his_uq_slope, big=True)
IndR_his_uq_slope_ens_mask = ca.MME_mask(IndR_his_uq_slope, big=True)

nIndR_his_vq_slope_ens_mask = ca.MME_mask(nIndR_his_vq_slope, big=True)
sIndR_his_vq_slope_ens_mask = ca.MME_mask(sIndR_his_vq_slope, big=True)
IndR_his_vq_slope_ens_mask = ca.MME_mask(IndR_his_vq_slope, big=True)

nIndR_his_uqvqdiv_slope_ens_mask = ca.MME_mask(nIndR_his_uqvqdiv_slope, big=True)
sIndR_his_uqvqdiv_slope_ens_mask = ca.MME_mask(sIndR_his_uqvqdiv_slope, big=True)
IndR_his_uqvqdiv_slope_ens_mask = ca.MME_mask(IndR_his_uqvqdiv_slope, big=True)

# nIndR_his_div_slope_ens_mask = ca.MME_mask(nIndR_his_div_slope, big=True)
# sIndR_his_div_slope_ens_mask = ca.MME_mask(sIndR_his_div_slope, big=True)
# IndR_his_div_slope_ens_mask = ca.MME_mask(IndR_his_div_slope, big=True)

nIndR_his_sstDJF_slope_ens_mask = ca.MME_mask(nIndR_his_sstDJF_slope, big=True)
sIndR_his_sstDJF_slope_ens_mask = ca.MME_mask(sIndR_his_sstDJF_slope, big=True)
IndR_his_sstDJF_slope_ens_mask = ca.MME_mask(IndR_his_sstDJF_slope, big=True)

nIndR_his_sstMAM_slope_ens_mask = ca.MME_mask(nIndR_his_sstMAM_slope, big=True)
sIndR_his_sstMAM_slope_ens_mask = ca.MME_mask(sIndR_his_sstMAM_slope, big=True)
IndR_his_sstMAM_slope_ens_mask = ca.MME_mask(IndR_his_sstMAM_slope, big=True)

nIndR_his_sstJJA_slope_ens_mask = ca.MME_mask(nIndR_his_sstJJA_slope, big=True)
sIndR_his_sstJJA_slope_ens_mask = ca.MME_mask(sIndR_his_sstJJA_slope, big=True)
IndR_his_sstJJA_slope_ens_mask = ca.MME_mask(IndR_his_sstJJA_slope, big=True)

nIndR_his_sstSON_slope_ens_mask = ca.MME_mask(nIndR_his_sstSON_slope, big=True)
sIndR_his_sstSON_slope_ens_mask = ca.MME_mask(sIndR_his_sstSON_slope, big=True)
IndR_his_sstSON_slope_ens_mask = ca.MME_mask(IndR_his_sstSON_slope, big=True)

nIndR_his_sstnDJF_slope_ens_mask = ca.MME_mask(nIndR_his_sstnDJF_slope, big=True)
sIndR_his_sstnDJF_slope_ens_mask = ca.MME_mask(sIndR_his_sstnDJF_slope, big=True)
IndR_his_sstnDJF_slope_ens_mask = ca.MME_mask(IndR_his_sstnDJF_slope, big=True)

In [ ]:
nIndR_his_wind_ens_mask = ca.wind_check(
    xr.where(nIndR_his_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_his_v_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_his_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_his_v_slope_ens_mask > 0.0, 1.0, 0.0),
)
sIndR_his_wind_ens_mask = ca.wind_check(
    xr.where(sIndR_his_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_his_v_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_his_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_his_v_slope_ens_mask > 0.0, 1.0, 0.0),
)
IndR_his_wind_ens_mask = ca.wind_check(
    xr.where(IndR_his_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_his_v_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_his_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_his_v_slope_ens_mask > 0.0, 1.0, 0.0),
)

nIndR_his_uqvq_ens_mask = ca.wind_check(
    xr.where(nIndR_his_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_his_vq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_his_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_his_vq_slope_ens_mask > 0.0, 1.0, 0.0),
)
sIndR_his_uqvq_ens_mask = ca.wind_check(
    xr.where(sIndR_his_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_his_vq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_his_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_his_vq_slope_ens_mask > 0.0, 1.0, 0.0),
)
IndR_his_uqvq_ens_mask = ca.wind_check(
    xr.where(IndR_his_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_his_vq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_his_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_his_vq_slope_ens_mask > 0.0, 1.0, 0.0),
)

### ssp585

In [ ]:
nIndR_ssp585_p3_wind_mask = ca.wind_check(
    xr.where(nIndR_ssp585_p3_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_v_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_v_pvalue <= 0.05, 1.0, 0.0),
)
sIndR_ssp585_p3_wind_mask = ca.wind_check(
    xr.where(sIndR_ssp585_p3_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_v_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_v_pvalue <= 0.05, 1.0, 0.0),
)
IndR_ssp585_p3_wind_mask = ca.wind_check(
    xr.where(IndR_ssp585_p3_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_v_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_u_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_v_pvalue <= 0.05, 1.0, 0.0),
)

nIndR_ssp585_p3_uqvq_mask = ca.wind_check(
    xr.where(nIndR_ssp585_p3_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_vq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_vq_pvalue <= 0.05, 1.0, 0.0),
)
sIndR_ssp585_p3_uqvq_mask = ca.wind_check(
    xr.where(sIndR_ssp585_p3_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_vq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_vq_pvalue <= 0.05, 1.0, 0.0),
)
IndR_ssp585_p3_uqvq_mask = ca.wind_check(
    xr.where(IndR_ssp585_p3_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_vq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_uq_pvalue <= 0.05, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_vq_pvalue <= 0.05, 1.0, 0.0),
)

### ssp585集合平均

In [ ]:
nIndR_ssp585_p3_hgt_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_hgt_slope, big=True)
sIndR_ssp585_p3_hgt_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_hgt_slope, big=True)
IndR_ssp585_p3_hgt_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_hgt_slope, big=True)

nIndR_ssp585_p3_u_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_u_slope, big=True)
sIndR_ssp585_p3_u_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_u_slope, big=True)
IndR_ssp585_p3_u_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_u_slope, big=True)

nIndR_ssp585_p3_v_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_v_slope, big=True)
sIndR_ssp585_p3_v_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_v_slope, big=True)
IndR_ssp585_p3_v_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_v_slope, big=True)

nIndR_ssp585_p3_pre_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_pre_slope, big=True)
sIndR_ssp585_p3_pre_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_pre_slope, big=True)
IndR_ssp585_p3_pre_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_pre_slope, big=True)

nIndR_ssp585_p3_uq_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_uq_slope, big=True)
sIndR_ssp585_p3_uq_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_uq_slope, big=True)
IndR_ssp585_p3_uq_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_uq_slope, big=True)

nIndR_ssp585_p3_vq_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_vq_slope, big=True)
sIndR_ssp585_p3_vq_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_vq_slope, big=True)
IndR_ssp585_p3_vq_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_vq_slope, big=True)

nIndR_ssp585_p3_uqvqdiv_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_uqvqdiv_slope, big=True)
sIndR_ssp585_p3_uqvqdiv_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_uqvqdiv_slope, big=True)
IndR_ssp585_p3_uqvqdiv_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_uqvqdiv_slope, big=True)

# nIndR_ssp585_p3_div_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_div_slope, big=True)
# sIndR_ssp585_p3_div_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_div_slope, big=True)
# IndR_ssp585_p3_div_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_div_slope, big=True)

nIndR_ssp585_p3_sstDJF_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_sstDJF_slope, big=True)
sIndR_ssp585_p3_sstDJF_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_sstDJF_slope, big=True)
IndR_ssp585_p3_sstDJF_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_sstDJF_slope, big=True)

nIndR_ssp585_p3_sstMAM_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_sstMAM_slope, big=True)
sIndR_ssp585_p3_sstMAM_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_sstMAM_slope, big=True)
IndR_ssp585_p3_sstMAM_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_sstMAM_slope, big=True)

nIndR_ssp585_p3_sstJJA_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_sstJJA_slope, big=True)
sIndR_ssp585_p3_sstJJA_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_sstJJA_slope, big=True)
IndR_ssp585_p3_sstJJA_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_sstJJA_slope, big=True)

nIndR_ssp585_p3_sstSON_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_sstSON_slope, big=True)
sIndR_ssp585_p3_sstSON_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_sstSON_slope, big=True)
IndR_ssp585_p3_sstSON_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_sstSON_slope, big=True)

nIndR_ssp585_p3_sstnDJF_slope_ens_mask = ca.MME_mask(nIndR_ssp585_p3_sstnDJF_slope, big=True)
sIndR_ssp585_p3_sstnDJF_slope_ens_mask = ca.MME_mask(sIndR_ssp585_p3_sstnDJF_slope, big=True)
IndR_ssp585_p3_sstnDJF_slope_ens_mask = ca.MME_mask(IndR_ssp585_p3_sstnDJF_slope, big=True)

In [ ]:
nIndR_ssp585_p3_wind_ens_mask = ca.wind_check(
    xr.where(nIndR_ssp585_p3_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_v_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_v_slope_ens_mask > 0.0, 1.0, 0.0),
)
sIndR_ssp585_p3_wind_ens_mask = ca.wind_check(
    xr.where(sIndR_ssp585_p3_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_v_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_v_slope_ens_mask > 0.0, 1.0, 0.0),
)
IndR_ssp585_p3_wind_ens_mask = ca.wind_check(
    xr.where(IndR_ssp585_p3_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_v_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_v_slope_ens_mask > 0.0, 1.0, 0.0),
)

nIndR_ssp585_p3_uqvq_ens_mask = ca.wind_check(
    xr.where(nIndR_ssp585_p3_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_vq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_ssp585_p3_vq_slope_ens_mask > 0.0, 1.0, 0.0),
)
sIndR_ssp585_p3_uqvq_ens_mask = ca.wind_check(
    xr.where(sIndR_ssp585_p3_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_vq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_ssp585_p3_vq_slope_ens_mask > 0.0, 1.0, 0.0),
)
IndR_ssp585_p3_uqvq_ens_mask = ca.wind_check(
    xr.where(IndR_ssp585_p3_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_vq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_ssp585_p3_vq_slope_ens_mask > 0.0, 1.0, 0.0),
)

### difference 集合平均

In [ ]:
IndR_diff_pre_slope_ens_mask = ca.MME_mask(IndR_diff_pre_slope, big=True)
IndR_diff_hgt_slope_ens_mask = ca.MME_mask(IndR_diff_hgt_slope, big=True)
IndR_diff_u_slope_ens_mask = ca.MME_mask(IndR_diff_u_slope, big=True)
IndR_diff_v_slope_ens_mask = ca.MME_mask(IndR_diff_v_slope, big=True)
IndR_diff_sstDJF_slope_ens_mask = ca.MME_mask(IndR_diff_sstDJF_slope, big=True)
IndR_diff_sstMAM_slope_ens_mask = ca.MME_mask(IndR_diff_sstMAM_slope, big=True)
IndR_diff_sstJJA_slope_ens_mask = ca.MME_mask(IndR_diff_sstJJA_slope, big=True)
IndR_diff_sstSON_slope_ens_mask = ca.MME_mask(IndR_diff_sstSON_slope, big=True)
IndR_diff_sstnDJF_slope_ens_mask = ca.MME_mask(IndR_diff_sstnDJF_slope, big=True)
IndR_diff_uq_slope_ens_mask = ca.MME_mask(IndR_diff_uq_slope, big=True)
IndR_diff_vq_slope_ens_mask = ca.MME_mask(IndR_diff_vq_slope, big=True)
IndR_diff_uqvqdiv_slope_ens_mask = ca.MME_mask(IndR_diff_uqvqdiv_slope, big=True)

In [ ]:
nIndR_diff_pre_slope_ens_mask = ca.MME_mask(nIndR_diff_pre_slope, big=True)
nIndR_diff_hgt_slope_ens_mask = ca.MME_mask(nIndR_diff_hgt_slope, big=True)
nIndR_diff_u_slope_ens_mask = ca.MME_mask(nIndR_diff_u_slope, big=True)
nIndR_diff_v_slope_ens_mask = ca.MME_mask(nIndR_diff_v_slope, big=True)
nIndR_diff_sstDJF_slope_ens_mask = ca.MME_mask(nIndR_diff_sstDJF_slope, big=True)
nIndR_diff_sstMAM_slope_ens_mask = ca.MME_mask(nIndR_diff_sstMAM_slope, big=True)
nIndR_diff_sstJJA_slope_ens_mask = ca.MME_mask(nIndR_diff_sstJJA_slope, big=True)
nIndR_diff_sstSON_slope_ens_mask = ca.MME_mask(nIndR_diff_sstSON_slope, big=True)
nIndR_diff_sstnDJF_slope_ens_mask = ca.MME_mask(nIndR_diff_sstnDJF_slope, big=True)
nIndR_diff_uq_slope_ens_mask = ca.MME_mask(nIndR_diff_uq_slope, big=True)
nIndR_diff_vq_slope_ens_mask = ca.MME_mask(nIndR_diff_vq_slope, big=True)
nIndR_diff_uqvqdiv_slope_ens_mask = ca.MME_mask(nIndR_diff_uqvqdiv_slope, big=True)

In [ ]:
sIndR_diff_pre_slope_ens_mask = ca.MME_mask(sIndR_diff_pre_slope, big=True)
sIndR_diff_hgt_slope_ens_mask = ca.MME_mask(sIndR_diff_hgt_slope, big=True)
sIndR_diff_u_slope_ens_mask = ca.MME_mask(sIndR_diff_u_slope, big=True)
sIndR_diff_v_slope_ens_mask = ca.MME_mask(sIndR_diff_v_slope, big=True)
sIndR_diff_sstDJF_slope_ens_mask = ca.MME_mask(sIndR_diff_sstDJF_slope, big=True)
sIndR_diff_sstMAM_slope_ens_mask = ca.MME_mask(sIndR_diff_sstMAM_slope, big=True)
sIndR_diff_sstJJA_slope_ens_mask = ca.MME_mask(sIndR_diff_sstJJA_slope, big=True)
sIndR_diff_sstSON_slope_ens_mask = ca.MME_mask(sIndR_diff_sstSON_slope, big=True)
sIndR_diff_sstnDJF_slope_ens_mask = ca.MME_mask(sIndR_diff_sstnDJF_slope, big=True)
sIndR_diff_uq_slope_ens_mask = ca.MME_mask(sIndR_diff_uq_slope, big=True)
sIndR_diff_vq_slope_ens_mask = ca.MME_mask(sIndR_diff_vq_slope, big=True)
sIndR_diff_uqvqdiv_slope_ens_mask = ca.MME_mask(sIndR_diff_uqvqdiv_slope, big=True)

In [ ]:
nIndR_diff_wind_ens_mask = ca.wind_check(
    xr.where(nIndR_diff_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_diff_v_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_diff_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_diff_v_slope_ens_mask > 0.0, 1.0, 0.0),
)
sIndR_diff_wind_ens_mask = ca.wind_check(
    xr.where(sIndR_diff_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_diff_v_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_diff_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_diff_v_slope_ens_mask > 0.0, 1.0, 0.0),
)
IndR_diff_wind_ens_mask = ca.wind_check(
    xr.where(IndR_diff_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_diff_v_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_diff_u_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_diff_v_slope_ens_mask > 0.0, 1.0, 0.0),
)

nIndR_diff_uqvq_ens_mask = ca.wind_check(
    xr.where(nIndR_diff_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_diff_vq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_diff_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(nIndR_diff_vq_slope_ens_mask > 0.0, 1.0, 0.0),
)
sIndR_diff_uqvq_ens_mask = ca.wind_check(
    xr.where(sIndR_diff_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_diff_vq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_diff_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(sIndR_diff_vq_slope_ens_mask > 0.0, 1.0, 0.0),
)
IndR_diff_uqvq_ens_mask = ca.wind_check(
    xr.where(IndR_diff_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_diff_vq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_diff_uq_slope_ens_mask > 0.0, 1.0, 0.0),
    xr.where(IndR_diff_vq_slope_ens_mask > 0.0, 1.0, 0.0),
)

## 画图部分

### fig.1 降水

##### ensemble

In [ ]:
pplt.rc.reset()
pplt.rc.grid = False
# pplt.rc.reset()
cl = 0  # 设置地图投影的中心纬度
proj = pplt.PlateCarree(central_longitude=cl)

fig_pre_ens = pplt.figure(span=False, share=False, refwidth=4.0, wspace=4.0, hspace=4.5, outerpad=2.0)
nrows = 3
ncols = 3
plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
axs = fig_pre_ens.subplots(plot_array, proj=proj)

#   set the geo_ticks and map projection to the plots
xticks = np.array([60, 90, 120])  # 设置纬度刻度
yticks = np.arange(0, 46, 15)  # 设置经度刻度
# 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
# 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
extents = [50.0, 140.0, yticks[0], 45.0]
sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=10)

# ===================================================
ski = 2
n = 1
w, h = 0.12, 0.14
# ===================================================
for ax in axs:
    # NC area
    x0 = NC_W
    y0 = NC_S
    width = NC_E-NC_W
    height = NC_N-NC_S
    sepl.patches(ax, x0 - cl, y0, width, height, proj, linewidth=1.2)
# ===================================================
for ax in axs[:,0]:
    # India
    x0 = India_W
    y0 = India_S
    width = India_E-India_W
    height = India_N-India_S
    sepl.patches(ax, x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
for ax in axs[:,1]:
    # northern India
    x0 = nIndia_W
    y0 = nIndia_S
    width = nIndia_E-nIndia_W
    height = nIndia_N-nIndia_S
    sepl.patches(ax, x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
for ax in axs[:,2]:
    # northern India
    x0 = sIndia_W
    y0 = sIndia_S
    width = sIndia_E-sIndia_W
    height = sIndia_N-sIndia_S
    sepl.patches(ax, x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
# ensemble
con = axs[0,0].contourf(
    IndR_his_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both"
    )
sepl.plt_sig(
    IndR_his_pre_slope_ens, axs[0,0], n, np.where(IndR_his_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[0,0].format(
    ltitle="1979-2014 ens", rtitle="SASMR",
)
# ===================================================
con = axs[0,1].contourf(
    nIndR_his_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    nIndR_his_pre_slope_ens, axs[0,1], n, np.where(nIndR_his_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[0,1].format(
    ltitle="1979-2014 ens", rtitle="northern SASMR",
)
# ===================================================
con = axs[0,2].contourf(
    sIndR_his_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    sIndR_his_pre_slope_ens, axs[0,2], n, np.where(sIndR_his_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[0,2].format(
    ltitle="1979-2014 ens", rtitle="southern SASMR",
)
# ===================================================
con = axs[1,0].contourf(
    IndR_ssp585_p3_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both"
    )
sepl.plt_sig(
    IndR_ssp585_p3_pre_slope_ens, axs[1,0], n, np.where(IndR_ssp585_p3_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[1,0].format(
    ltitle="2064-2099 ens", rtitle="SASMR",
)
# ===================================================
con = axs[1,1].contourf(
    nIndR_ssp585_p3_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    nIndR_ssp585_p3_pre_slope_ens, axs[1,1], n, np.where(nIndR_ssp585_p3_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[1,1].format(
    ltitle="2064-2099 ens", rtitle="northern SASMR",
)
# ===================================================
con = axs[1,2].contourf(
    sIndR_ssp585_p3_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    sIndR_ssp585_p3_pre_slope_ens, axs[1,2], n, np.where(sIndR_ssp585_p3_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[1,2].format(
    ltitle="2064-2099 ens", rtitle="southern SASMR",
)
# ===================================================
# difference
con = axs[2,0].contourf(
    IndR_diff_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both"
    )
sepl.plt_sig(
    IndR_diff_pre_slope_ens, axs[2,0], n, np.where(IndR_diff_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[2,0].format(
    ltitle="diff ens", rtitle="SASMR",
)
# ===================================================
con = axs[2,1].contourf(
    nIndR_diff_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    nIndR_diff_pre_slope_ens, axs[2,1], n, np.where(nIndR_diff_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[2,1].format(
    ltitle="diff ens", rtitle="northern SASMR",
)
# ===================================================
con = axs[2,2].contourf(
    sIndR_diff_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    sIndR_diff_pre_slope_ens, axs[2,2], n, np.where(sIndR_diff_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[2,2].format(
    ltitle="diff ens", rtitle="southern SASMR",
)
# ===================================================

cb = fig_pre_ens.colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8)
cb.set_ticks(np.arange(-2.0,2.1, 0.4))
axs.format(linewidth=1.2)
fig_pre_ens.format(abc="(a)", abcloc="l", rc_kw={"grid": False, "reso": "lo", "figure.facecolor":"white", "font.large": 13})
pplt.rc.reset()

##### historical ensemble

In [ ]:
pplt.rc.reset()
pplt.rc.grid = False
# pplt.rc.reset()
cl = 0  # 设置地图投影的中心纬度
proj = pplt.PlateCarree(central_longitude=cl)

fig_pre_ens = pplt.figure(span=False, share=False, refwidth=4.0, wspace=4.0, hspace=4.5, outerpad=2.0)
nrows = 3
ncols = 1
plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
axs = fig_pre_ens.subplots(plot_array, proj=proj)

#   set the geo_ticks and map projection to the plots
xticks = np.array([60, 90, 120])  # 设置纬度刻度
yticks = np.arange(0, 46, 15)  # 设置经度刻度
# 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
# 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
extents = [50.0, 140.0, yticks[0], 45.0]
sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=10)

# ===================================================
ski = 2
n = 1
w, h = 0.12, 0.14
# ===================================================
for ax in axs:
    # NC area
    x0 = NC_W
    y0 = NC_S
    width = NC_E-NC_W
    height = NC_N-NC_S
    sepl.patches(ax, x0 - cl, y0, width, height, proj, linewidth=1.2)
# ===================================================
# India
x0 = India_W
y0 = India_S
width = India_E-India_W
height = India_N-India_S
sepl.patches(axs[0], x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
# northern India
x0 = nIndia_W
y0 = nIndia_S
width = nIndia_E-nIndia_W
height = nIndia_N-nIndia_S
sepl.patches(axs[1], x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
# northern India
x0 = sIndia_W
y0 = sIndia_S
width = sIndia_E-sIndia_W
height = sIndia_N-sIndia_S
sepl.patches(axs[2], x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
# ensemble
con = axs[0,0].contourf(
    IndR_his_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both"
    )
sepl.plt_sig(
    IndR_his_pre_slope_ens, axs[0,0], n, np.where(IndR_his_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[0,0].format(
    ltitle="1979-2014 ens", rtitle="SASMR",
)
# ===================================================
con = axs[1,0].contourf(
    nIndR_his_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    nIndR_his_pre_slope_ens, axs[1,0], n, np.where(nIndR_his_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[1,0].format(
    ltitle="1979-2014 ens", rtitle="northern SASMR",
)
# ===================================================
con = axs[2,0].contourf(
    sIndR_his_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    sIndR_his_pre_slope_ens, axs[2,0], n, np.where(sIndR_his_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[2,0].format(
    ltitle="1979-2014 ens", rtitle="southern SASMR",
)
# ===================================================

cb = fig_pre_ens.colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8)
cb.set_ticks(np.arange(-2.0,2.1, 0.4))
axs.format(linewidth=1.2)
fig_pre_ens.format(abc="(a)", abcloc="l", rc_kw={"grid": False, "reso": "lo", "figure.facecolor":"white", "font.large": 13})
pplt.rc.reset()

##### ssp585 ensemble

In [ ]:
pplt.rc.reset()
pplt.rc.grid = False
# pplt.rc.reset()
cl = 0  # 设置地图投影的中心纬度
proj = pplt.PlateCarree(central_longitude=cl)

fig_pre_ens = pplt.figure(span=False, share=False, refwidth=4.0, wspace=4.0, hspace=4.5, outerpad=2.0)
nrows = 3
ncols = 1
plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
axs = fig_pre_ens.subplots(plot_array, proj=proj)

#   set the geo_ticks and map projection to the plots
xticks = np.array([60, 90, 120])  # 设置纬度刻度
yticks = np.arange(0, 46, 15)  # 设置经度刻度
# 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
# 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
extents = [50.0, 140.0, yticks[0], 45.0]
sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=10)

# ===================================================
ski = 2
n = 1
w, h = 0.12, 0.14
# ===================================================
for ax in axs:
    # NC area
    x0 = NC_W
    y0 = NC_S
    width = NC_E-NC_W
    height = NC_N-NC_S
    sepl.patches(ax, x0 - cl, y0, width, height, proj, linewidth=1.2)
# ===================================================
# India
x0 = India_W
y0 = India_S
width = India_E-India_W
height = India_N-India_S
sepl.patches(axs[0], x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
# northern India
x0 = nIndia_W
y0 = nIndia_S
width = nIndia_E-nIndia_W
height = nIndia_N-nIndia_S
sepl.patches(axs[1], x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
# northern India
x0 = sIndia_W
y0 = sIndia_S
width = sIndia_E-sIndia_W
height = sIndia_N-sIndia_S
sepl.patches(axs[2], x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
# ensemble
con = axs[0].contourf(
    IndR_ssp585_p3_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both"
    )
sepl.plt_sig(
    IndR_ssp585_p3_pre_slope_ens, axs[0], n, np.where(IndR_ssp585_p3_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[0].format(
    ltitle="2064-2099 ens", rtitle="SASMR",
)
# ===================================================
con = axs[1].contourf(
    nIndR_ssp585_p3_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    nIndR_ssp585_p3_pre_slope_ens, axs[1], n, np.where(nIndR_ssp585_p3_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[1].format(
    ltitle="2064-2099 ens", rtitle="northern SASMR",
)
# ===================================================
con = axs[2].contourf(
    sIndR_ssp585_p3_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    sIndR_ssp585_p3_pre_slope_ens, axs[2], n, np.where(sIndR_ssp585_p3_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[2].format(
    ltitle="2064-2099 ens", rtitle="southern SASMR",
)
# ===================================================
cb = fig_pre_ens.colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8)
cb.set_ticks(np.arange(-2.0,2.1, 0.4))
axs.format(linewidth=1.2)
fig_pre_ens.format(abc="(a)", abcloc="l", rc_kw={"grid": False, "reso": "lo", "figure.facecolor":"white", "font.large": 13})
pplt.rc.reset()

##### diff ens

In [ ]:
pplt.rc.reset()
pplt.rc.grid = False
# pplt.rc.reset()
cl = 0  # 设置地图投影的中心纬度
proj = pplt.PlateCarree(central_longitude=cl)

fig_pre_ens = pplt.figure(span=False, share=False, refwidth=4.0, wspace=4.0, hspace=4.5, outerpad=2.0)
nrows = 3
ncols = 1
plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
axs = fig_pre_ens.subplots(plot_array, proj=proj)

#   set the geo_ticks and map projection to the plots
xticks = np.array([60, 90, 120])  # 设置纬度刻度
yticks = np.arange(0, 46, 15)  # 设置经度刻度
# 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
# 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
extents = [50.0, 140.0, yticks[0], 45.0]
sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=10)

# ===================================================
ski = 2
n = 1
w, h = 0.12, 0.14
# ===================================================
for ax in axs:
    # NC area
    x0 = NC_W
    y0 = NC_S
    width = NC_E-NC_W
    height = NC_N-NC_S
    sepl.patches(ax, x0 - cl, y0, width, height, proj, linewidth=1.2)
# ===================================================
# India
x0 = India_W
y0 = India_S
width = India_E-India_W
height = India_N-India_S
sepl.patches(axs[0], x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
# northern India
x0 = nIndia_W
y0 = nIndia_S
width = nIndia_E-nIndia_W
height = nIndia_N-nIndia_S
sepl.patches(axs[1], x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
# northern India
x0 = sIndia_W
y0 = sIndia_S
width = sIndia_E-sIndia_W
height = sIndia_N-sIndia_S
sepl.patches(axs[2], x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
# ===================================================
# difference
con = axs[0].contourf(
    IndR_diff_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both"
    )
sepl.plt_sig(
    IndR_diff_pre_slope_ens, axs[0], n, np.where(IndR_diff_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[0].format(
    ltitle="diff ens", rtitle="SASMR",
)
# ===================================================
con = axs[1].contourf(
    nIndR_diff_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    nIndR_diff_pre_slope_ens, axs[1], n, np.where(nIndR_diff_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[1].format(
    ltitle="diff ens", rtitle="northern SASMR",
)
# ===================================================
con = axs[2].contourf(
    sIndR_diff_pre_slope_ens,
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both",
    )
sepl.plt_sig(
    sIndR_diff_pre_slope_ens, axs[2], n, np.where(sIndR_diff_pre_slope_ens_mask[::n, ::n] > 0.0), "bright purple", 12.0,
)

axs[2].format(
    ltitle="diff ens", rtitle="southern SASMR",
)
# ===================================================

cb = fig_pre_ens.colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8)
cb.set_ticks(np.arange(-2.0,2.1, 0.4))
axs.format(linewidth=1.2)
fig_pre_ens.format(abc="(a)", abcloc="l", rc_kw={"grid": False, "reso": "lo", "figure.facecolor":"white", "font.large": 13})
pplt.rc.reset()

##### 单个模式

In [ ]:
# individual models
for num_mod, mod in enumerate(models.data):
  pplt.rc.reset()
  pplt.rc.grid = False
  # pplt.rc.reset()
  cl = 0  # 设置地图投影的中心纬度
  proj = pplt.PlateCarree(central_longitude=cl)

  fig_pre_individual_models = pplt.figure(span=False, share=False, refwidth=4.0, wspace=4.0, hspace=4.5, outerpad=2.0)
  nrows = 3
  ncols = 3
  plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
  axs = fig_pre_individual_models.subplots(plot_array, proj=proj)

  #   set the geo_ticks and map projection to the plots
  xticks = np.array([60, 90, 120])  # 设置纬度刻度
  yticks = np.arange(0, 46, 15)  # 设置经度刻度
  # 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
  # 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
  extents = [50.0, 140.0, yticks[0], 45.0]
  sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=10)

  # ===================================================
  ski = 2
  n = 1
  w, h = 0.12, 0.14
  # ===================================================
  for ax in axs:
      # NC area
      x0 = NC_W
      y0 = NC_S
      width = NC_E-NC_W
      height = NC_N-NC_S
      sepl.patches(ax, x0 - cl, y0, width, height, proj, linewidth=1.2)
  # ===================================================
  for ax in axs[:,0]:
      # India
      x0 = India_W
      y0 = India_S
      width = India_E-India_W
      height = India_N-India_S
      sepl.patches(ax, x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
  # ===================================================
  for ax in axs[:,1]:
      # northern India
      x0 = nIndia_W
      y0 = nIndia_S
      width = nIndia_E-nIndia_W
      height = nIndia_N-nIndia_S
      sepl.patches(ax, x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
  # ===================================================
  for ax in axs[:,2]:
      # northern India
      x0 = sIndia_W
      y0 = sIndia_S
      width = sIndia_E-sIndia_W
      height = sIndia_N-sIndia_S
      sepl.patches(ax, x0 - cl, y0, width, height, proj, linestyle="-", linewidth=1.2)
  # ===================================================
  con = axs[0,0].contourf(
    IndR_his_pre_slope.sel(models=mod),
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both"
    )
  sepl.plt_sig(
      IndR_his_pre_slope.sel(models=mod), axs[0,0], n, np.where(IndR_his_pre_pvalue.sel(models=mod)[::n, ::n] <= 0.10), "bright purple", 12.0,
  )

  axs[0,0].format(
      ltitle="1979-2014 {}".format(mod), rtitle="SASMR",
  )
  # ===================================================
  con = axs[0,1].contourf(
      nIndR_his_pre_slope.sel(models=mod),
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94},
      levels=np.arange(-1.6,1.7,0.2),
      zorder=0.8,
      extend="both",
      )
  sepl.plt_sig(
      nIndR_his_pre_slope.sel(models=mod), axs[0,1], n, np.where(nIndR_his_pre_pvalue.sel(models=mod)[::n, ::n] <= 0.10), "bright purple", 12.0,
  )

  axs[0,1].format(
      ltitle="1979-2014 {}".format(mod), rtitle="northern SASMR",
  )
  # ===================================================
  con = axs[0,2].contourf(
      sIndR_his_pre_slope.sel(models=mod),
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94},
      levels=np.arange(-1.6,1.7,0.2),
      zorder=0.8,
      extend="both",
      )
  sepl.plt_sig(
      sIndR_his_pre_slope.sel(models=mod), axs[0,2], n, np.where(sIndR_his_pre_pvalue.sel(models=mod)[::n, ::n] <= 0.10), "bright purple", 12.0,
  )

  axs[0,2].format(
      ltitle="1979-2014 {}".format(mod), rtitle="southern SASMR",
  )
  # ===================================================
  con = axs[1,0].contourf(
    IndR_ssp585_p3_pre_slope.sel(models=mod),
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both"
    )
  sepl.plt_sig(
      IndR_ssp585_p3_pre_slope.sel(models=mod), axs[1,0], n, np.where(IndR_ssp585_p3_pre_pvalue.sel(models=mod)[::n, ::n] <= 0.10), "bright purple", 12.0,
  )

  axs[1,0].format(
      ltitle="2064-2099 {}".format(mod), rtitle="SASMR",
  )
  # ===================================================
  con = axs[1,1].contourf(
      nIndR_ssp585_p3_pre_slope.sel(models=mod),
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94},
      levels=np.arange(-1.6,1.7,0.2),
      zorder=0.8,
      extend="both",
      )
  sepl.plt_sig(
      nIndR_ssp585_p3_pre_slope.sel(models=mod), axs[1,1], n, np.where(nIndR_ssp585_p3_pre_pvalue.sel(models=mod)[::n, ::n] <= 0.10), "bright purple", 12.0,
  )

  axs[1,1].format(
      ltitle="2064-2099 {}".format(mod), rtitle="northern SASMR",
  )
  # ===================================================
  con = axs[1,2].contourf(
      sIndR_ssp585_p3_pre_slope.sel(models=mod),
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94},
      levels=np.arange(-1.6,1.7,0.2),
      zorder=0.8,
      extend="both",
      )
  # sepl.plt_sig(
  #     sIndR_ssp585_p3_pre_slope.sel(models=mod), axs[1,2], n, np.where(sIndR_ssp585_p3_pre_pvalue.sel(models=mod)[::n, ::n] <= 0.10), "bright purple", 12.0,
  # )

  axs[1,2].format(
      ltitle="2064-2099 {}".format(mod), rtitle="southern SASMR",
  )
  # ===================================================
  con = axs[2,0].contourf(
    IndR_diff_pre_slope.sel(models=mod),
    cmap="ColdHot",
    cmap_kw={"left": 0.06, "right": 0.94},
    levels=np.arange(-1.6,1.7,0.2),
    zorder=0.8,
    extend="both"
    )
  # sepl.plt_sig(
  #     IndR_diff_pre_slope.sel(models=mod), axs[2,0], n, np.where(IndR_diff_pre_slope_ens_mask.sel(models=mod)[::n, ::n] > 0.0), "bright purple", 12.0,
  # )

  axs[2,0].format(
      ltitle="diff {}".format(mod), rtitle="SASMR",
  )
  # ===================================================
  con = axs[2,1].contourf(
      nIndR_diff_pre_slope.sel(models=mod),
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94},
      levels=np.arange(-1.6,1.7,0.2),
      zorder=0.8,
      extend="both",
      )
  # sepl.plt_sig(
  #     nIndR_diff_pre_slope.sel(models=mod), axs[2,1], n, np.where(nIndR_diff_pre_slope_ens_mask.sel(models=mod)[::n, ::n] > 0.0), "bright purple", 12.0,
  # )

  axs[2,1].format(
      ltitle="diff {}".format(mod), rtitle="northern SASMR",
  )
  # ===================================================
  con = axs[2,2].contourf(
      sIndR_diff_pre_slope.sel(models=mod),
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94},
      levels=np.arange(-1.6,1.7,0.2),
      zorder=0.8,
      extend="both",
      )
  # sepl.plt_sig(
  #     sIndR_diff_pre_slope.sel(models=mod), axs[2,2], n, np.where(sIndR_diff_pre_slope_ens_mask.sel(models=mod)[::n, ::n] > 0.0), "bright purple", 12.0,
  # )

  axs[2,2].format(
      ltitle="diff {}".format(mod), rtitle="southern SASMR",
  )
  cb = fig_pre_individual_models.colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8)
  cb.set_ticks(np.arange(-2.0,2.1, 0.4))
  axs.format(linewidth=1.2)
  fig_pre_individual_models.format(abc="(a)", abcloc="l", rc_kw={"grid": False, "reso": "lo", "figure.facecolor":"white", "font.large": 13})
  fig_pre_individual_models.save("/home/ys17-23/chenhj/0302code/CMIP6/divided_SASMR_detrend/fig_pre_individul_{}.png".format(mod),dpi=150)
  del(fig_pre_individual_models)
  pplt.rc.reset()

### fig.2 环流场

#### historical ensemble

In [ ]:
pplt.rc.reset()
pplt.rc.grid = False
startlevel=[-15, -8, -6]
spacinglevel=[1.5, 0.8, 0.6]
scalelevel=[0.30, 0.17, 0.14]

cl = 0  # 设置地图投影的中心纬度
proj = pplt.PlateCarree(central_longitude=cl)

fig_his_cir_ens = pplt.figure(span=False, share=False, refwidth=6.0, wspace=6.0, hspace=5.5, outerpad=2.0)
nrows = 3
ncols = 3
plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
# plot_array[-1,-1] = 0
axs = fig_his_cir_ens.subplots(plot_array, proj=proj)

#   set the geo_ticks and map projection to the plots
# xticks = np.array([30, 60, 90, 120, 150, 180])  # 设置纬度刻度
xticks = np.array([60, 90, 120])  # 设置纬度刻度
yticks = np.arange(0, 46, 15)  # 设置经度刻度
# 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
# 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
extents = [40.0, 140.0, yticks[0], 55.0]
sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=14, lonminorspace=5, coastlinewidth=1.7, majorticklabelpad=3.0, majorticklen=6.0, minorticklen=5.0)
# ===================================================
ski = 2
n = 1
w, h = 0.12, 0.14
# ======================================
for ax in axs:
    rect = Rectangle((1 - w, 0), w, h, transform=ax.transAxes, fc="white", ec="k", lw=1.3, zorder=1.1)
    ax.add_patch(rect)
for num_lev,lev in enumerate([200.0, 500.0, 850.0]):
    con = axs[num_lev, 0].contourf(
        IndR_his_hgt_slope_ens.sel(level=lev),
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        IndR_his_hgt_slope_ens.sel(level=lev), axs[num_lev, 0], n, np.where(IndR_his_hgt_slope_ens_mask.sel(level=lev)[::n, ::n] > 0.0), "bright purple", 19.0,
    )
    # axs[num_lev, 0].quiver(
    #     IndRAIR_his_u_slope.sel(level=lev)[::ski, ::ski],
    #     IndRAIR_his_v_slope.sel(level=lev)[::ski, ::ski],
    #     zorder=1.1,
    #     headwidth=2.6,
    #     headlength=2.3,
    #     headaxislength=2.3,
    #     scale_units="xy",
    #     scale=scalelevel[num_lev],
    #     pivot="mid",
    #     color="grey6",
    # )

    m = axs[num_lev, 0].quiver(
        IndR_his_u_slope_ens.sel(level=lev).where(IndR_his_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        IndR_his_v_slope_ens.sel(level=lev).where(IndR_his_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        zorder=1.1,
        headwidth=2.6,
        headlength=2.3,
        headaxislength=2.3,
        minlength=0.5,
        width=0.004,
        scale_units="xy",
        scale=scalelevel[num_lev],
        pivot="mid",
        color="black",
    )

    qk = axs[num_lev, 0].quiverkey(
        m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
    )
    axs[num_lev, 0].format(
        ltitle="1979-2014 {:.0f} hPa ens".format(lev), rtitle="SASMR",
    )
    # ======================================
    con = axs[num_lev, 1].contourf(
        nIndR_his_hgt_slope_ens.sel(level=lev),
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        nIndR_his_hgt_slope_ens.sel(level=lev), axs[num_lev, 1], n, np.where(nIndR_his_hgt_slope_ens_mask.sel(level=lev)[::n, ::n] > 0.0), "bright purple", 19.0,
    )
    # axs[num_lev, 1].quiver(
    #     IndR_his_u_slope_ens.sel(level=lev)[::ski, ::ski],
    #     IndR_his_v_slope_ens.sel(level=lev)[::ski, ::ski],
    #     zorder=1.1,
    #     headwidth=2.6,
    #     headlength=2.3,
    #     headaxislength=2.3,
    #     scale_units="xy",
    #     scale=scalelevel[num_lev],
    #     pivot="mid",
    #     color="grey6",
    # )

    m = axs[num_lev, 1].quiver(
        nIndR_his_u_slope_ens.sel(level=lev).where(nIndR_his_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        nIndR_his_v_slope_ens.sel(level=lev).where(nIndR_his_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        zorder=1.1,
        headwidth=2.6,
        headlength=2.3,
        headaxislength=2.3,
        minlength=0.5,
        width=0.004,
        scale_units="xy",
        scale=scalelevel[num_lev],
        pivot="mid",
        color="black",
    )

    qk = axs[num_lev, 1].quiverkey(
        m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
    )
    axs[num_lev, 1].format(
        ltitle="1979-2014 {:.0f} hPa ens".format(lev), rtitle="northern SASMR",
    )
    # ======================================
    con = axs[num_lev, 2].contourf(
        sIndR_his_hgt_slope_ens.sel(level=lev),
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        sIndR_his_hgt_slope_ens.sel(level=lev), axs[num_lev, 2], n, np.where(sIndR_his_hgt_slope_ens_mask.sel(level=lev)[::n, ::n] > 0.0), "bright purple", 19.0,
    )
    # axs[num_lev, 2].quiver(
    #     IndR_his_u_slope_ens.sel(level=lev)[::ski, ::ski],
    #     IndR_his_v_slope_ens.sel(level=lev)[::ski, ::ski],
    #     zorder=1.1,
    #     headwidth=2.6,
    #     headlength=2.3,
    #     headaxislength=2.3,
    #     scale_units="xy",
    #     scale=scalelevel[num_lev],
    #     pivot="mid",
    #     color="grey6",
    # )

    m = axs[num_lev, 2].quiver(
        sIndR_his_u_slope_ens.sel(level=lev).where(sIndR_his_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        sIndR_his_v_slope_ens.sel(level=lev).where(sIndR_his_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        zorder=1.1,
        headwidth=2.6,
        headlength=2.3,
        headaxislength=2.3,
        minlength=0.5,
        width=0.004,
        scale_units="xy",
        scale=scalelevel[num_lev],
        pivot="mid",
        color="black",
    )

    qk = axs[num_lev, 2].quiverkey(
        m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
    )
    axs[num_lev, 2].format(
        ltitle="1979-2014 {:.0f} hPa ens".format(lev), rtitle="southern SASMR",
    )
    cb = axs[num_lev, 2].colorbar(con, loc="r", width=0.17, length=0.8, label="", ticklabelsize=12, pad=2.5)
    cb.set_ticks(np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]*2))
# ======================================
axs.format(linewidth=1.2, titlepad=7.0)
fig_his_cir_ens.format(abc="(a)", abcloc="l", rc_kw={"grid": False, "reso":"lo", "figure.facecolor":"white", "font.large":15})
pplt.rc.reset()

#### ssp585 ensemble

In [ ]:
pplt.rc.reset()
pplt.rc.grid = False
startlevel=[-15, -8, -6]
spacinglevel=[1.5, 0.8, 0.6]
scalelevel=[0.30, 0.17, 0.14]

cl = 0  # 设置地图投影的中心纬度
proj = pplt.PlateCarree(central_longitude=cl)

fig_ssp585_p3_cir_ens = pplt.figure(span=False, share=False, refwidth=6.0, wspace=6.0, hspace=5.5, outerpad=2.0)
nrows = 3
ncols = 3
plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
# plot_array[-1,-1] = 0
axs = fig_ssp585_p3_cir_ens.subplots(plot_array, proj=proj)

#   set the geo_ticks and map projection to the plots
# xticks = np.array([30, 60, 90, 120, 150, 180])  # 设置纬度刻度
xticks = np.array([60, 90, 120])  # 设置纬度刻度
yticks = np.arange(0, 46, 15)  # 设置经度刻度
# 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
# 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
extents = [40.0, 140.0, yticks[0], 55.0]
sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=14, lonminorspace=5, coastlinewidth=1.7, majorticklabelpad=3.0, majorticklen=6.0, minorticklen=5.0)
# ===================================================
ski = 2
n = 1
w, h = 0.12, 0.14
# ======================================
for ax in axs:
    rect = Rectangle((1 - w, 0), w, h, transform=ax.transAxes, fc="white", ec="k", lw=1.3, zorder=1.1)
    ax.add_patch(rect)
for num_lev,lev in enumerate([200.0, 500.0, 850.0]):
    con = axs[num_lev, 0].contourf(
        IndR_ssp585_p3_hgt_slope_ens.sel(level=lev),
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        IndR_ssp585_p3_hgt_slope_ens.sel(level=lev), axs[num_lev, 0], n, np.where(IndR_ssp585_p3_hgt_slope_ens_mask.sel(level=lev)[::n, ::n] > 0.0), "bright purple", 19.0,
    )
    # axs[num_lev, 0].quiver(
    #     IndRAIR_ssp585_p3_u_slope.sel(level=lev)[::ski, ::ski],
    #     IndRAIR_ssp585_p3_v_slope.sel(level=lev)[::ski, ::ski],
    #     zorder=1.1,
    #     headwidth=2.6,
    #     headlength=2.3,
    #     headaxislength=2.3,
    #     scale_units="xy",
    #     scale=scalelevel[num_lev],
    #     pivot="mid",
    #     color="grey6",
    # )

    m = axs[num_lev, 0].quiver(
        IndR_ssp585_p3_u_slope_ens.sel(level=lev).where(IndR_ssp585_p3_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        IndR_ssp585_p3_v_slope_ens.sel(level=lev).where(IndR_ssp585_p3_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        zorder=1.1,
        headwidth=2.6,
        headlength=2.3,
        headaxislength=2.3,
        minlength=0.5,
        width=0.004,
        scale_units="xy",
        scale=scalelevel[num_lev],
        pivot="mid",
        color="black",
    )

    qk = axs[num_lev, 0].quiverkey(
        m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
    )
    axs[num_lev, 0].format(
        ltitle="2064-2099 {:.0f} hPa ens".format(lev), rtitle="SASMR",
    )
    # ======================================
    con = axs[num_lev, 1].contourf(
        nIndR_ssp585_p3_hgt_slope_ens.sel(level=lev),
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        nIndR_ssp585_p3_hgt_slope_ens.sel(level=lev), axs[num_lev, 1], n, np.where(nIndR_ssp585_p3_hgt_slope_ens_mask.sel(level=lev)[::n, ::n] > 0.0), "bright purple", 19.0,
    )
    # axs[num_lev, 1].quiver(
    #     IndR_ssp585_p3_u_slope_ens.sel(level=lev)[::ski, ::ski],
    #     IndR_ssp585_p3_v_slope_ens.sel(level=lev)[::ski, ::ski],
    #     zorder=1.1,
    #     headwidth=2.6,
    #     headlength=2.3,
    #     headaxislength=2.3,
    #     scale_units="xy",
    #     scale=scalelevel[num_lev],
    #     pivot="mid",
    #     color="grey6",
    # )

    m = axs[num_lev, 1].quiver(
        nIndR_ssp585_p3_u_slope_ens.sel(level=lev).where(nIndR_ssp585_p3_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        nIndR_ssp585_p3_v_slope_ens.sel(level=lev).where(nIndR_ssp585_p3_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        zorder=1.1,
        headwidth=2.6,
        headlength=2.3,
        headaxislength=2.3,
        minlength=0.5,
        width=0.004,
        scale_units="xy",
        scale=scalelevel[num_lev],
        pivot="mid",
        color="black",
    )

    qk = axs[num_lev, 1].quiverkey(
        m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
    )
    axs[num_lev, 1].format(
        ltitle="2064-2099 {:.0f} hPa ens".format(lev), rtitle="northern SASMR",
    )
    # ======================================
    con = axs[num_lev, 2].contourf(
        sIndR_ssp585_p3_hgt_slope_ens.sel(level=lev),
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        sIndR_ssp585_p3_hgt_slope_ens.sel(level=lev), axs[num_lev, 2], n, np.where(sIndR_ssp585_p3_hgt_slope_ens_mask.sel(level=lev)[::n, ::n] > 0.0), "bright purple", 19.0,
    )
    # axs[num_lev, 2].quiver(
    #     IndR_ssp585_p3_u_slope_ens.sel(level=lev)[::ski, ::ski],
    #     IndR_ssp585_p3_v_slope_ens.sel(level=lev)[::ski, ::ski],
    #     zorder=1.1,
    #     headwidth=2.6,
    #     headlength=2.3,
    #     headaxislength=2.3,
    #     scale_units="xy",
    #     scale=scalelevel[num_lev],
    #     pivot="mid",
    #     color="grey6",
    # )

    m = axs[num_lev, 2].quiver(
        sIndR_ssp585_p3_u_slope_ens.sel(level=lev).where(sIndR_ssp585_p3_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        sIndR_ssp585_p3_v_slope_ens.sel(level=lev).where(sIndR_ssp585_p3_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        zorder=1.1,
        headwidth=2.6,
        headlength=2.3,
        headaxislength=2.3,
        minlength=0.5,
        width=0.004,
        scale_units="xy",
        scale=scalelevel[num_lev],
        pivot="mid",
        color="black",
    )

    qk = axs[num_lev, 2].quiverkey(
        m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
    )
    axs[num_lev, 2].format(
        ltitle="2064-2099 {:.0f} hPa ens".format(lev), rtitle="southern SASMR",
    )
    cb = axs[num_lev, 2].colorbar(con, loc="r", width=0.17, length=0.8, label="", ticklabelsize=12, pad=2.5)
    cb.set_ticks(np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]*2))
# ======================================
axs.format(linewidth=1.2, titlepad=7.0)
fig_ssp585_p3_cir_ens.format(abc="(a)", abcloc="l", rc_kw={"grid": False, "reso":"lo", "figure.facecolor":"white", "font.large":15})
pplt.rc.reset()

#### diff ensemble

In [ ]:
pplt.rc.reset()
pplt.rc.grid = False
startlevel=[-15, -8, -6]
spacinglevel=[1.5, 0.8, 0.6]
scalelevel=[0.30, 0.17, 0.14]

cl = 0  # 设置地图投影的中心纬度
proj = pplt.PlateCarree(central_longitude=cl)

fig_diff_cir_ens = pplt.figure(span=False, share=False, refwidth=6.0, wspace=6.0, hspace=5.5, outerpad=2.0)
nrows = 3
ncols = 3
plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
# plot_array[-1,-1] = 0
axs = fig_diff_cir_ens.subplots(plot_array, proj=proj)

#   set the geo_ticks and map projection to the plots
# xticks = np.array([30, 60, 90, 120, 150, 180])  # 设置纬度刻度
xticks = np.array([60, 90, 120])  # 设置纬度刻度
yticks = np.arange(0, 46, 15)  # 设置经度刻度
# 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
# 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
extents = [40.0, 140.0, yticks[0], 55.0]
sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=14, lonminorspace=5, coastlinewidth=1.7, majorticklabelpad=3.0, majorticklen=6.0, minorticklen=5.0)
# ===================================================
ski = 2
n = 1
w, h = 0.12, 0.14
# ======================================
for ax in axs:
    rect = Rectangle((1 - w, 0), w, h, transform=ax.transAxes, fc="white", ec="k", lw=1.3, zorder=1.1)
    ax.add_patch(rect)
for num_lev,lev in enumerate([200.0, 500.0, 850.0]):
    con = axs[num_lev, 0].contourf(
        IndR_diff_hgt_slope_ens.sel(level=lev),
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        IndR_diff_hgt_slope_ens.sel(level=lev), axs[num_lev, 0], n, np.where(IndR_diff_hgt_slope_ens_mask.sel(level=lev)[::n, ::n] > 0.0), "bright purple", 19.0,
    )
    m = axs[num_lev, 0].quiver(
        IndR_diff_u_slope_ens.sel(level=lev).where(IndR_diff_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        IndR_diff_v_slope_ens.sel(level=lev).where(IndR_diff_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        zorder=1.1,
        headwidth=2.6,
        headlength=2.3,
        headaxislength=2.3,
        minlength=0.5,
        width=0.004,
        scale_units="xy",
        scale=scalelevel[num_lev],
        pivot="mid",
        color="black",
    )

    qk = axs[num_lev, 0].quiverkey(
        m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
    )
    axs[num_lev, 0].format(
        ltitle="diff {:.0f} hPa ens".format(lev), rtitle="SASMR",
    )
    # ======================================
    con = axs[num_lev, 1].contourf(
        nIndR_diff_hgt_slope_ens.sel(level=lev),
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        nIndR_diff_hgt_slope_ens.sel(level=lev), axs[num_lev, 1], n, np.where(nIndR_diff_hgt_slope_ens_mask.sel(level=lev)[::n, ::n] > 0.0), "bright purple", 19.0,
    )
    # axs[num_lev, 1].quiver(
    #     IndR_diff_u_slope_ens.sel(level=lev)[::ski, ::ski],
    #     IndR_diff_v_slope_ens.sel(level=lev)[::ski, ::ski],
    #     zorder=1.1,
    #     headwidth=2.6,
    #     headlength=2.3,
    #     headaxislength=2.3,
    #     scale_units="xy",
    #     scale=scalelevel[num_lev],
    #     pivot="mid",
    #     color="grey6",
    # )

    m = axs[num_lev, 1].quiver(
        nIndR_diff_u_slope_ens.sel(level=lev).where(nIndR_diff_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        nIndR_diff_v_slope_ens.sel(level=lev).where(nIndR_diff_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        zorder=1.1,
        headwidth=2.6,
        headlength=2.3,
        headaxislength=2.3,
        minlength=0.5,
        width=0.004,
        scale_units="xy",
        scale=scalelevel[num_lev],
        pivot="mid",
        color="black",
    )

    qk = axs[num_lev, 1].quiverkey(
        m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
    )
    axs[num_lev, 1].format(
        ltitle="diff {:.0f} hPa ens".format(lev), rtitle="northern SASMR",
    )
    # ======================================
    con = axs[num_lev, 2].contourf(
        sIndR_diff_hgt_slope_ens.sel(level=lev),
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        sIndR_diff_hgt_slope_ens.sel(level=lev), axs[num_lev, 2], n, np.where(sIndR_diff_hgt_slope_ens_mask.sel(level=lev)[::n, ::n] > 0.0), "bright purple", 19.0,
    )
    # axs[num_lev, 2].quiver(
    #     IndR_diff_u_slope_ens.sel(level=lev)[::ski, ::ski],
    #     IndR_diff_v_slope_ens.sel(level=lev)[::ski, ::ski],
    #     zorder=1.1,
    #     headwidth=2.6,
    #     headlength=2.3,
    #     headaxislength=2.3,
    #     scale_units="xy",
    #     scale=scalelevel[num_lev],
    #     pivot="mid",
    #     color="grey6",
    # )

    m = axs[num_lev, 2].quiver(
        sIndR_diff_u_slope_ens.sel(level=lev).where(sIndR_diff_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        sIndR_diff_v_slope_ens.sel(level=lev).where(sIndR_diff_wind_ens_mask.sel(level=lev) > 0.0)[::ski, ::ski],
        zorder=1.1,
        headwidth=2.6,
        headlength=2.3,
        headaxislength=2.3,
        minlength=0.5,
        width=0.004,
        scale_units="xy",
        scale=scalelevel[num_lev],
        pivot="mid",
        color="black",
    )

    qk = axs[num_lev, 2].quiverkey(
        m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
    )
    axs[num_lev, 2].format(
        ltitle="diff {:.0f} hPa ens".format(lev), rtitle="southern SASMR",
    )
    cb = axs[num_lev, 2].colorbar(con, loc="r", width=0.17, length=0.8, label="", ticklabelsize
                                  =12, pad=2.5)
    cb.set_ticks(np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]*2))
# ======================================
axs.format(linewidth=1.2, titlepad=7.0)
fig_diff_cir_ens.format(abc="(a)", abcloc="l", rc_kw={"grid": False, "reso":"lo", "figure.facecolor":"white", "font.large":15})
pplt.rc.reset()

#### fig.2 sup 单个模式

In [ ]:
for num_mod, mod in enumerate(models.data):
  pplt.rc.reset()
  pplt.rc.grid = False
  startlevel=[-15, -8, -6]
  spacinglevel=[1.5, 0.8, 0.6]
  scalelevel=[0.30, 0.17, 0.14]

  cl = 0  # 设置地图投影的中心纬度
  proj = pplt.PlateCarree(central_longitude=cl)

  fig_his_cir_individual = pplt.figure(span=False, share=False, refwidth=6.0, wspace=6.0, hspace=5.5, outerpad=2.0)
  nrows = 3
  ncols = 3
  plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
  # plot_array[-1,-1] = 0
  axs = fig_his_cir_individual.subplots(plot_array, proj=proj)

  #   set the geo_ticks and map projection to the plots
  # xticks = np.array([30, 60, 90, 120, 150, 180])  # 设置纬度刻度
  xticks = np.array([60, 90, 120])  # 设置纬度刻度
  yticks = np.arange(0, 46, 15)  # 设置经度刻度
  # 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
  # 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
  extents = [40.0, 140.0, yticks[0], 55.0]
  sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=14, lonminorspace=5, coastlinewidth=1.7, majorticklabelpad=3.0, majorticklen=6.0, minorticklen=5.0)
  # ===================================================
  ski = 2
  n = 1
  w, h = 0.12, 0.14
  # ======================================
  for ax in axs:
      rect = Rectangle((1 - w, 0), w, h, transform=ax.transAxes, fc="white", ec="k", lw=1.3, zorder=1.1)
      ax.add_patch(rect)
  for num_lev,lev in enumerate([200.0, 500.0, 850.0]):
      con = axs[num_lev, 0].contourf(
          IndR_his_hgt_slope.sel(models=mod,level=lev),
          cmap="ColdHot",
          cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
          levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
          zorder=0.8,
          extend="both"
      )
      sepl.plt_sig(
          IndR_his_hgt_slope.sel(models=mod,level=lev), axs[num_lev, 0], n, np.where(IndR_his_hgt_pvalue.sel(models=mod,level=lev)[::n, ::n] <= 0.05), "bright purple", 19.0,
      )
      # axs[num_lev, 0].quiver(
      #     IndRAIR_his_u_slope.sel(models=mod,level=lev)[::ski, ::ski],
      #     IndRAIR_his_v_slope.sel(models=mod,level=lev)[::ski, ::ski],
      #     zorder=1.1,
      #     headwidth=2.6,
      #     headlength=2.3,
      #     headaxislength=2.3,
      #     scale_units="xy",
      #     scale=scalelevel[num_lev],
      #     pivot="mid",
      #     color="grey6",
      # )

      m = axs[num_lev, 0].quiver(
          IndR_his_u_slope.sel(models=mod,level=lev).where(IndR_his_wind_mask.sel(models=mod,level=lev) > 0.0)[::ski, ::ski],
          IndR_his_v_slope.sel(models=mod,level=lev).where(IndR_his_wind_mask.sel(models=mod,level=lev) > 0.0)[::ski, ::ski],
          zorder=1.1,
          headwidth=2.6,
          headlength=2.3,
          headaxislength=2.3,
          minlength=0.5,
          width=0.004,
          scale_units="xy",
          scale=scalelevel[num_lev],
          pivot="mid",
          color="black",
      )

      qk = axs[num_lev, 0].quiverkey(
          m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
      )
      axs[num_lev, 0].format(
          ltitle="1979-2014 {:.0f} hPa ens".format(lev), rtitle="SASMR",
      )
      # ======================================
      con = axs[num_lev, 1].contourf(
          nIndR_his_hgt_slope.sel(models=mod,level=lev),
          cmap="ColdHot",
          cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
          levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
          zorder=0.8,
          extend="both"
      )
      sepl.plt_sig(
          nIndR_his_hgt_slope.sel(models=mod,level=lev), axs[num_lev, 1], n, np.where(nIndR_his_hgt_pvalue.sel(models=mod,level=lev)[::n, ::n] <= 0.05), "bright purple", 19.0,
      )
      # axs[num_lev, 1].quiver(
      #     IndR_his_u_slope.sel(models=mod,level=lev)[::ski, ::ski],
      #     IndR_his_v_slope.sel(models=mod,level=lev)[::ski, ::ski],
      #     zorder=1.1,
      #     headwidth=2.6,
      #     headlength=2.3,
      #     headaxislength=2.3,
      #     scale_units="xy",
      #     scale=scalelevel[num_lev],
      #     pivot="mid",
      #     color="grey6",
      # )

      m = axs[num_lev, 1].quiver(
          nIndR_his_u_slope.sel(models=mod,level=lev).where(nIndR_his_wind_mask.sel(models=mod,level=lev) > 0.0)[::ski, ::ski],
          nIndR_his_v_slope.sel(models=mod,level=lev).where(nIndR_his_wind_mask.sel(models=mod,level=lev) > 0.0)[::ski, ::ski],
          zorder=1.1,
          headwidth=2.6,
          headlength=2.3,
          headaxislength=2.3,
          minlength=0.5,
          width=0.004,
          scale_units="xy",
          scale=scalelevel[num_lev],
          pivot="mid",
          color="black",
      )

      qk = axs[num_lev, 1].quiverkey(
          m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
      )
      axs[num_lev, 1].format(
          ltitle="1979-2014 {:.0f} hPa ens".format(lev), rtitle="northern SASMR",
      )
      # ======================================
      con = axs[num_lev, 2].contourf(
          sIndR_his_hgt_slope.sel(models=mod,level=lev),
          cmap="ColdHot",
          cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
          levels=np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]),
          zorder=0.8,
          extend="both"
      )
      sepl.plt_sig(
          sIndR_his_hgt_slope.sel(models=mod,level=lev), axs[num_lev, 2], n, np.where(sIndR_his_hgt_pvalue.sel(models=mod,level=lev)[::n, ::n] <= 0.05), "bright purple", 19.0,
      )
      # axs[num_lev, 2].quiver(
      #     IndR_his_u_slope.sel(models=mod,level=lev)[::ski, ::ski],
      #     IndR_his_v_slope.sel(models=mod,level=lev)[::ski, ::ski],
      #     zorder=1.1,
      #     headwidth=2.6,
      #     headlength=2.3,
      #     headaxislength=2.3,
      #     scale_units="xy",
      #     scale=scalelevel[num_lev],
      #     pivot="mid",
      #     color="grey6",
      # )

      m = axs[num_lev, 2].quiver(
          sIndR_his_u_slope.sel(models=mod,level=lev).where(sIndR_his_wind_mask.sel(models=mod,level=lev) > 0.0)[::ski, ::ski],
          sIndR_his_v_slope.sel(models=mod,level=lev).where(sIndR_his_wind_mask.sel(models=mod,level=lev) > 0.0)[::ski, ::ski],
          zorder=1.1,
          headwidth=2.6,
          headlength=2.3,
          headaxislength=2.3,
          minlength=0.5,
          width=0.004,
          scale_units="xy",
          scale=scalelevel[num_lev],
          pivot="mid",
          color="black",
      )

      qk = axs[num_lev, 2].quiverkey(
          m, X=1 - w / 2, Y=0.7 * h, U=0.5, label="0.5", labelpos="S", labelsep=0.10, fontproperties={"size": 10}, zorder=3.1,
      )
      axs[num_lev, 2].format(
          ltitle="1979-2014 {:.0f} hPa ens".format(lev), rtitle="southern SASMR",
      )
      cb = axs[num_lev, 2].colorbar(con, loc="r", width=0.17, length=0.8, label="", ticklabelsize=12, pad=2.5)
      cb.set_ticks(np.arange(startlevel[num_lev], -startlevel[num_lev]+spacinglevel[num_lev], spacinglevel[num_lev]*2))
  # ======================================
  axs.format(linewidth=1.2, titlepad=7.0)
  fig_his_cir_individual.format(abc="(a)", abcloc="l", rc_kw={"grid": False, "reso":"lo", "figure.facecolor":"white", "font.large":15})
  fig_his_cir_individual.save("/home/ys17-23/chenhj/0302code/CMIP6/divided_SASMR_detrend/fig_his_cir_individul_{}.png".format(mod),dpi=150)
  del(fig_his_cir_individual)
  pplt.rc.reset()

### fig.3 海温场

#### historical ensemble

In [ ]:
pplt.rc.reset()
pplt.rc.grid = False
startlevel=-6e-1
spacinglevel=0.06
cl = 180  # 设置地图投影的中心纬度
proj = pplt.PlateCarree(central_longitude=cl)

fig_sst_ens = pplt.figure(span=False, share=False, refwidth=4.0, wspace=4.0, hspace=3.8, outerpad=2.0)
nrows = 5
ncols = 3
plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
axs = fig_sst_ens.subplots(plot_array, proj=proj)

#   set the geo_ticks and map projection to the plots
# xticks = np.array([30, 60, 90, 120, 150, 180])  # 设置纬度刻度
xticks = np.array([30, 60, 90, 120, 150, 180, 210, 240, 270, 300])  # 设置纬度刻度
yticks = np.arange(0, 61, 30)  # 设置经度刻度
# 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
# 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
extents = [xticks[0], xticks[-1], -15, yticks[-1]]
sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=8, latminorspace=10, coastlinewidth=1.0)
# ===================================================
ski = 2
n = 2
w, h = 0.12, 0.14
# ======================================
for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
      [IndR_his_sstDJF_slope_ens, 
        IndR_his_sstMAM_slope_ens, 
        IndR_his_sstJJA_slope_ens, 
        IndR_his_sstSON_slope_ens, 
        IndR_his_sstnDJF_slope_ens], 
      [IndR_his_sstDJF_slope_ens_mask, 
        IndR_his_sstMAM_slope_ens_mask, 
        IndR_his_sstJJA_slope_ens_mask, 
        IndR_his_sstSON_slope_ens_mask, 
        IndR_his_sstnDJF_slope_ens_mask]):
  con = axs[i, 0].contourf(
      plot_slope,
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
      levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
      zorder=0.8,
      extend="both"
  )
  sepl.plt_sig(
      plot_slope, axs[i, 0], n, np.where(plot_pvalue[::n, ::n] > 0.0), "bright purple", 3.0,
  )

  axs[i, 0].format(
      ltitle="1979-2014 {} ens".format(season), rtitle="SASMR",
  )
# ======================================
for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
      [nIndR_his_sstDJF_slope_ens, 
      nIndR_his_sstMAM_slope_ens, 
      nIndR_his_sstJJA_slope_ens, 
      nIndR_his_sstSON_slope_ens, 
      nIndR_his_sstnDJF_slope_ens], 
      [nIndR_his_sstDJF_slope_ens_mask, 
      nIndR_his_sstMAM_slope_ens_mask, 
      nIndR_his_sstJJA_slope_ens_mask, 
      nIndR_his_sstSON_slope_ens_mask, 
      nIndR_his_sstnDJF_slope_ens_mask]):
  con = axs[i, 1].contourf(
      plot_slope,
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
      levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
      zorder=0.8,
      extend="both"
  )
  sepl.plt_sig(
      plot_slope, axs[i, 1], n, np.where(plot_pvalue[::n, ::n] > 0.0), "bright purple", 3.0,
  )

  axs[i, 1].format(
      ltitle="1979-2014 {} ens".format(season), rtitle="northern SASMR",
  )
# ======================================
for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
      [sIndR_his_sstDJF_slope_ens, 
      sIndR_his_sstMAM_slope_ens, 
      sIndR_his_sstJJA_slope_ens, 
      sIndR_his_sstSON_slope_ens, 
      sIndR_his_sstnDJF_slope_ens], 
      [sIndR_his_sstDJF_slope_ens_mask, 
      sIndR_his_sstMAM_slope_ens_mask, 
      sIndR_his_sstJJA_slope_ens_mask, 
      sIndR_his_sstSON_slope_ens_mask, 
      sIndR_his_sstnDJF_slope_ens_mask]):
  con = axs[i, 2].contourf(
      plot_slope,
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
      levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
      zorder=0.8,
      extend="both"
  )
  sepl.plt_sig(
      plot_slope, axs[i, 2], n, np.where(plot_pvalue[::n, ::n] > 0.0), "bright purple", 3.0,
  )

  axs[i, 2].format(
      ltitle="1979-2014 {} ens".format(season), rtitle="southern SASMR",
  )
# ======================================
cb = fig_sst_ens.colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8.0)
cb.set_ticks(np.arange(startlevel,-startlevel+0.01,0.24))
# # ======================================
# startlevel=-1.0
# spacinglevel=0.1
# # ======================================
# con = axs[3].contourf(
#     IndR_diff_sst_slope_gens,
#     cmap="ColdHot",
#     cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
#     levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
#     zorder=0.8,
#     extend="both"
# )
# sepl.plt_sig(
#     IndR_diff_sst_slope_gens, axs[3], n, np.where(IndR_diff_sst_slope_gens_mask[::n, ::n] > 0.00), "bright purple", 3.0,
# )
# axs[3].format(
#     ltitle="diff", rtitle="gMME",
# )
# cb = axs[3].colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8.0)
# cb.set_ticks(np.arange(-1.0,1.1,0.4))
# ======================================
axs.format(linewidth=1.2, titlepad=5.5)
fig_sst_ens.format(abc="(a)", abcloc="l", rc_kw={"figure.facecolor":"white", "font.large":9, "legend.fontsize":9, "grid":False, "reso":"lo"})
pplt.rc.reset()

#### ssp585 ensemble

In [ ]:
pplt.rc.reset()
pplt.rc.grid = False
startlevel=-6e-1
spacinglevel=0.06
cl = 180  # 设置地图投影的中心纬度
proj = pplt.PlateCarree(central_longitude=cl)

fig_sst_ens = pplt.figure(span=False, share=False, refwidth=4.0, wspace=4.0, hspace=3.8, outerpad=2.0)
nrows = 5
ncols = 3
plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
axs = fig_sst_ens.subplots(plot_array, proj=proj)

#   set the geo_ticks and map projection to the plots
# xticks = np.array([30, 60, 90, 120, 150, 180])  # 设置纬度刻度
xticks = np.array([30, 60, 90, 120, 150, 180, 210, 240, 270, 300])  # 设置纬度刻度
yticks = np.arange(0, 61, 30)  # 设置经度刻度
# 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
# 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
extents = [xticks[0], xticks[-1], -15, yticks[-1]]
sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=8, latminorspace=10, coastlinewidth=1.0)
# ===================================================
ski = 2
n = 2
w, h = 0.12, 0.14
# ======================================
for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
      [IndR_ssp585_p3_sstDJF_slope_ens, 
        IndR_ssp585_p3_sstMAM_slope_ens, 
        IndR_ssp585_p3_sstJJA_slope_ens, 
        IndR_ssp585_p3_sstSON_slope_ens, 
        IndR_ssp585_p3_sstnDJF_slope_ens], 
      [IndR_ssp585_p3_sstDJF_slope_ens_mask, 
        IndR_ssp585_p3_sstMAM_slope_ens_mask, 
        IndR_ssp585_p3_sstJJA_slope_ens_mask, 
        IndR_ssp585_p3_sstSON_slope_ens_mask, 
        IndR_ssp585_p3_sstnDJF_slope_ens_mask]):
  con = axs[i, 0].contourf(
      plot_slope,
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
      levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
      zorder=0.8,
      extend="both"
  )
  sepl.plt_sig(
      plot_slope, axs[i, 0], n, np.where(plot_pvalue[::n, ::n] > 0.0), "bright purple", 3.0,
  )

  axs[i, 0].format(
      ltitle="2064-2099 {} ens".format(season), rtitle="SASMR",
  )
# ======================================
for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
      [nIndR_ssp585_p3_sstDJF_slope_ens, 
      nIndR_ssp585_p3_sstMAM_slope_ens, 
      nIndR_ssp585_p3_sstJJA_slope_ens, 
      nIndR_ssp585_p3_sstSON_slope_ens, 
      nIndR_ssp585_p3_sstnDJF_slope_ens], 
      [nIndR_ssp585_p3_sstDJF_slope_ens_mask, 
      nIndR_ssp585_p3_sstMAM_slope_ens_mask, 
      nIndR_ssp585_p3_sstJJA_slope_ens_mask, 
      nIndR_ssp585_p3_sstSON_slope_ens_mask, 
      nIndR_ssp585_p3_sstnDJF_slope_ens_mask]):
  con = axs[i, 1].contourf(
      plot_slope,
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
      levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
      zorder=0.8,
      extend="both"
  )
  sepl.plt_sig(
      plot_slope, axs[i, 1], n, np.where(plot_pvalue[::n, ::n] > 0.0), "bright purple", 3.0,
  )

  axs[i, 1].format(
      ltitle="2064-2099 {} ens".format(season), rtitle="northern SASMR",
  )
# ======================================
for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
      [sIndR_ssp585_p3_sstDJF_slope_ens, 
      sIndR_ssp585_p3_sstMAM_slope_ens, 
      sIndR_ssp585_p3_sstJJA_slope_ens, 
      sIndR_ssp585_p3_sstSON_slope_ens, 
      sIndR_ssp585_p3_sstnDJF_slope_ens], 
      [sIndR_ssp585_p3_sstDJF_slope_ens_mask, 
      sIndR_ssp585_p3_sstMAM_slope_ens_mask, 
      sIndR_ssp585_p3_sstJJA_slope_ens_mask, 
      sIndR_ssp585_p3_sstSON_slope_ens_mask, 
      sIndR_ssp585_p3_sstnDJF_slope_ens_mask]):
  con = axs[i, 2].contourf(
      plot_slope,
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
      levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
      zorder=0.8,
      extend="both"
  )
  sepl.plt_sig(
      plot_slope, axs[i, 2], n, np.where(plot_pvalue[::n, ::n] > 0.0), "bright purple", 3.0,
  )

  axs[i, 2].format(
      ltitle="2064-2099 {} ens".format(season), rtitle="southern SASMR",
  )
# ======================================
cb = fig_sst_ens.colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8.0)
cb.set_ticks(np.arange(startlevel,-startlevel+0.01,0.24))
# # ======================================
# startlevel=-1.0
# spacinglevel=0.1
# # ======================================
# con = axs[3].contourf(
#     IndR_diff_sst_slope_gens,
#     cmap="ColdHot",
#     cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
#     levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
#     zorder=0.8,
#     extend="both"
# )
# sepl.plt_sig(
#     IndR_diff_sst_slope_gens, axs[3], n, np.where(IndR_diff_sst_slope_gens_mask[::n, ::n] > 0.00), "bright purple", 3.0,
# )
# axs[3].format(
#     ltitle="diff", rtitle="gMME",
# )
# cb = axs[3].colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8.0)
# cb.set_ticks(np.arange(-1.0,1.1,0.4))
# ======================================
axs.format(linewidth=1.2, titlepad=5.5)
fig_sst_ens.format(abc="(a)", abcloc="l", rc_kw={"figure.facecolor":"white", "font.large":9, "legend.fontsize":9, "grid":False, "reso":"lo"})
pplt.rc.reset()

#### diff ensemble

In [ ]:
pplt.rc.reset()
pplt.rc.grid = False
startlevel=-6e-1
spacinglevel=0.06
cl = 180  # 设置地图投影的中心纬度
proj = pplt.PlateCarree(central_longitude=cl)

fig_sst_ens = pplt.figure(span=False, share=False, refwidth=4.0, wspace=4.0, hspace=3.8, outerpad=2.0)
nrows = 5
ncols = 3
plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
axs = fig_sst_ens.subplots(plot_array, proj=proj)

#   set the geo_ticks and map projection to the plots
# xticks = np.array([30, 60, 90, 120, 150, 180])  # 设置纬度刻度
xticks = np.array([30, 60, 90, 120, 150, 180, 210, 240, 270, 300])  # 设置纬度刻度
yticks = np.arange(0, 61, 30)  # 设置经度刻度
# 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
# 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
extents = [xticks[0], xticks[-1], -15, yticks[-1]]
sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=8, latminorspace=10, coastlinewidth=1.0)
# ===================================================
ski = 2
n = 2
w, h = 0.12, 0.14
# ======================================
for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
      [IndR_diff_sstDJF_slope_ens, 
        IndR_diff_sstMAM_slope_ens, 
        IndR_diff_sstJJA_slope_ens, 
        IndR_diff_sstSON_slope_ens, 
        IndR_diff_sstnDJF_slope_ens], 
      [IndR_diff_sstDJF_slope_ens_mask, 
        IndR_diff_sstMAM_slope_ens_mask, 
        IndR_diff_sstJJA_slope_ens_mask, 
        IndR_diff_sstSON_slope_ens_mask, 
        IndR_diff_sstnDJF_slope_ens_mask]):
  con = axs[i, 0].contourf(
      plot_slope,
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
      levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
      zorder=0.8,
      extend="both"
  )
  sepl.plt_sig(
      plot_slope, axs[i, 0], n, np.where(plot_pvalue[::n, ::n] > 0.0), "bright purple", 3.0,
  )

  axs[i, 0].format(
      ltitle="diff {} ens".format(season), rtitle="SASMR",
  )
# ======================================
for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
      [nIndR_diff_sstDJF_slope_ens, 
      nIndR_diff_sstMAM_slope_ens, 
      nIndR_diff_sstJJA_slope_ens, 
      nIndR_diff_sstSON_slope_ens, 
      nIndR_diff_sstnDJF_slope_ens], 
      [nIndR_diff_sstDJF_slope_ens_mask, 
      nIndR_diff_sstMAM_slope_ens_mask, 
      nIndR_diff_sstJJA_slope_ens_mask, 
      nIndR_diff_sstSON_slope_ens_mask, 
      nIndR_diff_sstnDJF_slope_ens_mask]):
  con = axs[i, 1].contourf(
      plot_slope,
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
      levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
      zorder=0.8,
      extend="both"
  )
  sepl.plt_sig(
      plot_slope, axs[i, 1], n, np.where(plot_pvalue[::n, ::n] > 0.0), "bright purple", 3.0,
  )

  axs[i, 1].format(
      ltitle="diff {} ens".format(season), rtitle="northern SASMR",
  )
# ======================================
for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
      [sIndR_diff_sstDJF_slope_ens, 
      sIndR_diff_sstMAM_slope_ens, 
      sIndR_diff_sstJJA_slope_ens, 
      sIndR_diff_sstSON_slope_ens, 
      sIndR_diff_sstnDJF_slope_ens], 
      [sIndR_diff_sstDJF_slope_ens_mask, 
      sIndR_diff_sstMAM_slope_ens_mask, 
      sIndR_diff_sstJJA_slope_ens_mask, 
      sIndR_diff_sstSON_slope_ens_mask, 
      sIndR_diff_sstnDJF_slope_ens_mask]):
  con = axs[i, 2].contourf(
      plot_slope,
      cmap="ColdHot",
      cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
      levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
      zorder=0.8,
      extend="both"
  )
  sepl.plt_sig(
      plot_slope, axs[i, 2], n, np.where(plot_pvalue[::n, ::n] > 0.0), "bright purple", 3.0,
  )

  axs[i, 2].format(
      ltitle="diff {} ens".format(season), rtitle="southern SASMR",
  )
# ======================================
cb = fig_sst_ens.colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8.0)
cb.set_ticks(np.arange(startlevel,-startlevel+0.01,0.24))
# # ======================================
# startlevel=-1.0
# spacinglevel=0.1
# # ======================================
# con = axs[3].contourf(
#     IndR_diff_sst_slope_gens,
#     cmap="ColdHot",
#     cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
#     levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
#     zorder=0.8,
#     extend="both"
# )
# sepl.plt_sig(
#     IndR_diff_sst_slope_gens, axs[3], n, np.where(IndR_diff_sst_slope_gens_mask[::n, ::n] > 0.00), "bright purple", 3.0,
# )
# axs[3].format(
#     ltitle="diff", rtitle="gMME",
# )
# cb = axs[3].colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8.0)
# cb.set_ticks(np.arange(-1.0,1.1,0.4))
# ======================================
axs.format(linewidth=1.2, titlepad=5.5)
fig_sst_ens.format(abc="(a)", abcloc="l", rc_kw={"figure.facecolor":"white", "font.large":9, "legend.fontsize":9, "grid":False, "reso":"lo"})
pplt.rc.reset()

#### fig.3 sup 单个模式

In [ ]:
for num_mod, mod in enumerate(models.data):
  pplt.rc.reset()
  pplt.rc.grid = False
  startlevel=-6e-1
  spacinglevel=0.06
  cl = 180  # 设置地图投影的中心纬度
  proj = pplt.PlateCarree(central_longitude=cl)

  fig_his_sst_individual = pplt.figure(span=False, share=False, refwidth=4.0, wspace=4.0, hspace=3.8, outerpad=2.0)
  nrows = 5
  ncols = 3
  plot_array = np.reshape(range(1, nrows*ncols+1), (nrows, ncols))
  axs = fig_his_sst_individual.subplots(plot_array, proj=proj)

  #   set the geo_ticks and map projection to the plots
  # xticks = np.array([30, 60, 90, 120, 150, 180])  # 设置纬度刻度
  xticks = np.array([30, 60, 90, 120, 150, 180, 210, 240, 270, 300])  # 设置纬度刻度
  yticks = np.arange(0, 61, 30)  # 设置经度刻度
  # 设置绘图的经纬度范围extents，其中前两个参数为经度的最小值和最大值，后两个数为纬度的最小值和最大值
  # 当想要显示的经纬度范围不是正好等于刻度显示范围时，对extents进行相应的修改即可
  extents = [xticks[0], xticks[-1], -15, yticks[-1]]
  sepl.geo_ticks(axs, xticks, yticks, cl, extents, majorticklabelsize=8, latminorspace=10, coastlinewidth=1.0)
  # ===================================================
  ski = 2
  n = 2
  w, h = 0.12, 0.14
  # ======================================
  for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
        [IndR_his_sstDJF_slope.sel(models=mod), 
          IndR_his_sstMAM_slope.sel(models=mod), 
          IndR_his_sstJJA_slope.sel(models=mod), 
          IndR_his_sstSON_slope.sel(models=mod), 
          IndR_his_sstnDJF_slope.sel(models=mod)], 
        [IndR_his_sstDJF_pvalue.sel(models=mod), 
          IndR_his_sstMAM_pvalue.sel(models=mod), 
          IndR_his_sstJJA_pvalue.sel(models=mod), 
          IndR_his_sstSON_pvalue.sel(models=mod), 
          IndR_his_sstnDJF_pvalue.sel(models=mod)]):
    con = axs[i, 0].contourf(
        plot_slope,
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        plot_slope, axs[i, 0], n, np.where(plot_pvalue[::n, ::n] <= 0.05), "bright purple", 3.0,
    )

    axs[i, 0].format(
        ltitle="{} {}".format(season, mod), rtitle="SASMR",
    )
  # ======================================
  for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
        [nIndR_his_sstDJF_slope.sel(models=mod), 
        nIndR_his_sstMAM_slope.sel(models=mod), 
        nIndR_his_sstJJA_slope.sel(models=mod), 
        nIndR_his_sstSON_slope.sel(models=mod), 
        nIndR_his_sstnDJF_slope.sel(models=mod)], 
        [nIndR_his_sstDJF_pvalue.sel(models=mod), 
        nIndR_his_sstMAM_pvalue.sel(models=mod), 
        nIndR_his_sstJJA_pvalue.sel(models=mod), 
        nIndR_his_sstSON_pvalue.sel(models=mod), 
        nIndR_his_sstnDJF_pvalue.sel(models=mod)]):
    con = axs[i, 1].contourf(
        plot_slope,
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        plot_slope, axs[i, 1], n, np.where(plot_pvalue[::n, ::n] <= 0.05), "bright purple", 3.0,
    )

    axs[i, 1].format(
        ltitle="{} {}".format(season, mod), rtitle="northern SASMR",
    )
  # ======================================
  for (i, season), plot_slope, plot_pvalue in zip(enumerate(["DJF(-1)", "MAM(0)", "JJA(0)", "SON(0)", "DJ(0)F(1)"]), 
        [sIndR_his_sstDJF_slope.sel(models=mod), 
        sIndR_his_sstMAM_slope.sel(models=mod), 
        sIndR_his_sstJJA_slope.sel(models=mod), 
        sIndR_his_sstSON_slope.sel(models=mod), 
        sIndR_his_sstnDJF_slope.sel(models=mod)], 
        [sIndR_his_sstDJF_pvalue.sel(models=mod), 
        sIndR_his_sstMAM_pvalue.sel(models=mod), 
        sIndR_his_sstJJA_pvalue.sel(models=mod), 
        sIndR_his_sstSON_pvalue.sel(models=mod), 
        sIndR_his_sstnDJF_pvalue.sel(models=mod)]):
    con = axs[i, 2].contourf(
        plot_slope,
        cmap="ColdHot",
        cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
        levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
        zorder=0.8,
        extend="both"
    )
    sepl.plt_sig(
        plot_slope, axs[i, 2], n, np.where(plot_pvalue[::n, ::n] <= 0.05), "bright purple", 3.0,
    )

    axs[i, 2].format(
        ltitle="{} {}".format(season, mod), rtitle="southern SASMR",
    )
  # ======================================
  cb = fig_his_sst_individual.colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8.0)
  cb.set_ticks(np.arange(startlevel,-startlevel+0.01,0.24))
  # # ======================================
  # startlevel=-1.0
  # spacinglevel=0.1
  # # ======================================
  # con = axs[3].contourf(
  #     IndR_diff_sst_slope_gens,
  #     cmap="ColdHot",
  #     cmap_kw={"left": 0.06, "right": 0.94, "cut": -0.1},
  #     levels=np.arange(startlevel, -startlevel+spacinglevel, spacinglevel),
  #     zorder=0.8,
  #     extend="both"
  # )
  # sepl.plt_sig(
  #     IndR_diff_sst_slope_gens, axs[3], n, np.where(IndR_diff_sst_slope_gens_mask[::n, ::n] > 0.00), "bright purple", 3.0,
  # )
  # axs[3].format(
  #     ltitle="diff", rtitle="gMME",
  # )
  # cb = axs[3].colorbar(con, loc="b", width=0.13, length=0.7, label="", ticklabelsize=8.0)
  # cb.set_ticks(np.arange(-1.0,1.1,0.4))
  # ======================================
  axs.format(linewidth=1.2, titlepad=5.5)
  fig_his_sst_individual.format(abc="(a)", abcloc="l", rc_kw={"figure.facecolor":"white", "font.large":9, "legend.fontsize":9, "grid":False, "reso":"lo"})
  fig_his_sst_individual.save("/home/ys17-23/chenhj/0302code/CMIP6/divided_SASMR_detrend/fig_his_sst_individul_{}.png".format(mod),dpi=150)
  del(fig_his_sst_individual)
  pplt.rc.reset()